# Results and plots

In [ ]:
import os
import umap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.stats import ttest_ind, sem, t, fisher_exact
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches

np.random.seed(13)

In [ ]:
output_directory =  # COMPLETE HERE
plot_directory =  # COMPLETE HERE

# Load and format dataframes

In [ ]:
path = os.path.join(output_directory, "dataframes_v8", "merged_annotated.csv")
df_imported = pd.read_csv(path, sep=';', encoding="utf-8", low_memory=False)
print(df_imported.shape)
df_imported.head()

In [ ]:
df_imported = df_imported.loc[df_imported.loc[:, "nb_rna_out"] > 10, :]
print(df_imported.shape)

In [ ]:
df_kif1c = df_imported.loc[df_imported.loc[:, "gene"] == "KIF1C", :].copy()

def create_key(row):
    gene = row["gene"]
    puro = row["puromycin"]
    batch = int(row["experience"].split("_")[-1])
    if puro == 1:
        key = gene + "_puro_" + str(batch)
    else:
        key = gene + "_" + str(batch)
    return key

df_kif1c.loc[:, "key"] = df_kif1c.apply(create_key, axis=1)
df_kif1c.loc[:, "proportion_protrusion_30_area"] = df_kif1c.loc[:, "area_opening_30"] / df_kif1c.loc[:, "area_cyt"]

print(df_kif1c.shape)

In [ ]:
def create_key(row):
    gene = row["gene"]
    puro = row["puromycin"]
    drug = row["drug"]
    if drug == drug:
        key = gene + "_" + str(drug)
    elif puro == 1:
        key = gene + "_puro"
    else:
        key = gene
    return key

def create_label(row):
    exclusive_label = row["exclusive_label"]
    pattern_cell = row["pattern_cell"]
    pattern_p_bodies = row["pattern_p_bodies"]
    pattern_translation_factories = row["pattern_translation_factories"]
    pattern_intranuclear = row["pattern_intranuclear"]
    pattern_nuclear = row["pattern_nuclear"]
    pattern_perinuclear = row["pattern_perinuclear"]
    pattern_protrusion = row["pattern_protrusion"]
    pattern_random = row["pattern_random"]
    if exclusive_label != exclusive_label or not exclusive_label:
        return np.nan
    if pattern_cell:
        return "cell"
    elif pattern_p_bodies or pattern_translation_factories:
        return "foci"
    elif pattern_intranuclear:
        return "intranuclear"
    elif pattern_nuclear:
        return "nuclear"
    elif pattern_perinuclear:
        return "perinuclear"
    elif pattern_protrusion:
        return "protrusion"
    elif pattern_random:
        return "random"
    else:
        return np.nan
        
def gather_pattern_foci(row):
    pattern_p_bodies = row["pattern_p_bodies"]
    pattern_translation_factories = row["pattern_translation_factories"]
    annotated = row["annotated"]
    if annotated:
        if pattern_p_bodies or pattern_translation_factories:
            return True
        else:
            return False
    else:
        return np.nan

def fix_MYH3(row):
    gene = row["gene"]
    if gene == "MYH3H":
        return "MYH3"
    else:
        return gene
    
df = df_imported.copy()
df.loc[:, "gene"] = df.apply(fix_MYH3, axis=1)
df.loc[:, "key"] = df.apply(create_key, axis=1)
df.loc[:, "label"] = df.apply(create_label, axis=1)
df.loc[:, "pattern_foci"] = df.apply(gather_pattern_foci, axis=1)
df.loc[:, "proportion_protrusion_30_area"] = df.loc[:, "area_opening_30"] / df.loc[:, "area_cyt"]

print(df.shape)
df.head()

In [ ]:
df_30 = df.copy()

# remove old KIF1C experiments with puromycin
mask_experiences_kif1c_1 = df_30.loc[:, "experience"] == "KIF1C_unknown_nopuro_aubin_nodrug_1"
df_30 = df_30.loc[~mask_experiences_kif1c_1, :]
mask_experiences_kif1c_puro_1 = df_30.loc[:, "experience"] == "KIF1C_unknown_puro_racha_nodrug_1"
df_30 = df_30.loc[~mask_experiences_kif1c_puro_1, :]

# remove drug (DMSO and LG007)
mask_no_drug = df_30.drug.isna()
df_30_no_drug = df_30.loc[mask_no_drug, :].copy()

print(df_30.shape)
print(df_30_no_drug.shape)

# Functions and parameters

## General boxplots

In [ ]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

def get_nb_cells(data, genes_distribution):
    # get number of cells
    x_labels = []
    for key in genes_distribution:
        nb_cells = int(data.loc[data.key == key, ["id_cell", "key"]].groupby("key").count().sum())
        if "puro" in key:
            key = key.split("_")[0] + "*"
        new_label = key
        x_labels.append(new_label)
    x_ticks = [i for i in range(len(x_labels))]
    
    return x_ticks, x_labels

def get_whiskers_values(data, feature):
    # get whiskers values
    max_value = data.loc[:, feature].max()
    min_value = data.loc[:, feature].min()
    df_whiskers = data.loc[:, ["key", feature]].groupby(by="key").agg([q1, q3]).stack(level=0)
    df_whiskers["IQR"] = df_whiskers["q3"] - df_whiskers["q1"]
    df_whiskers["whiskers_top"] = df_whiskers["q3"] + df_whiskers["IQR"] * 1.5
    df_whiskers["whiskers_bottom"] = df_whiskers["q1"] - df_whiskers["IQR"] * 1.5
    max_whiskers_top = min(df_whiskers["whiskers_top"].max(), max_value)
    min_whiskers_bottom = max(df_whiskers["whiskers_bottom"].min(), min_value)
    marge = 5 * max_whiskers_top / 100
    
    return max_whiskers_top, min_whiskers_bottom, marge

def plot_boxplot(data, feature, genes_distribution, ax, show_whisker, show_flier, random_flier):
    # boxplot
    boxprops = dict(linestyle='-', linewidth=2, edgecolor='black')
    flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
    medianprops = dict(linestyle='-', linewidth=2, color='black')
    meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
    capprops = dict(linestyle='-', linewidth=1.5, color='grey')
    whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
    if show_whisker:
        sns.boxplot(x="key", y=feature, data=data, order=genes_distribution, ax=ax,
                    color="#4daf4a",
                    showmeans=True, meanline=False, meanprops=meanprops,
                    boxprops=boxprops,
                    showfliers=show_flier, flierprops=flierprops, 
                    medianprops=medianprops, 
                    capprops=capprops,
                    whiskerprops=whiskerprops, whis=1.5)
    else:
        sns.boxplot(x="key", y=feature, data=data, order=genes_distribution, ax=ax,
                    color="#4daf4a",
                    showmeans=True, meanline=False, meanprops=meanprops,
                    boxprops=boxprops,
                    showfliers=show_flier, flierprops=flierprops, 
                    medianprops=medianprops, 
                    showcaps=False, capprops=capprops,
                    whiskerprops=whiskerprops, whis=0)

    # scatter plot
    if random_flier:
        data_ = data.query("key in {0}".format(genes_distribution))
        int_gene = {}
        for i, gene in enumerate(genes_distribution):
            int_gene[gene] = i
        values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
        values_x = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_x
        values_y = data_.loc[:, feature]
        plt.scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.1)
        
    return

def format_axes(x_ticks, x_labels, ylim, min_whiskers_bottom, max_whiskers_top, marge, rotation_x=90):
    # axes
    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=rotation_x, fontweight="bold", fontsize=15)
    plt.yticks(fontweight="bold", fontsize=15)
    plt.xlabel("")
    plt.ylabel("")
    if ylim is "auto":
        plt.ylim(min_whiskers_bottom - marge, max_whiskers_top + marge)
    elif isinstance(ylim, tuple):
        plt.ylim(ylim[0], ylim[1])
        
    return

def save_frame(path_output, extension):
    # save frame
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
    
    return

In [ ]:
def plot_boxplot_feature(feature, data, figsize=(15, 5),
                         ylim="auto", horizontal_line=None,
                         show_whisker=False, show_flier=False, random_flier=True,
                         path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro"]
    nuclear_edge = ["SPEN", "ASPM", "ASPM_puro"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["MYH3", "MYH3_puro", "CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro",
                  "RAB13",  "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (random + protrusion + nuclear_edge + perinuclear + intranuclear 
                          + p_bodies + translation_factory)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(
        data=data, 
        genes_distribution=genes_distribution)
    
    # get whiskers values
    max_whiskers_top, min_whiskers_bottom, marge = get_whiskers_values(
        data=data, 
        feature=feature)

    # plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # boxplot
    plot_boxplot(
        data=data, 
        feature=feature, 
        genes_distribution=genes_distribution, 
        ax=ax, 
        show_whisker=show_whisker, 
        show_flier=show_flier, 
        random_flier=random_flier)
        
    # text and lines
    plt.axvline(x=4.5, c="red")
    plt.axvline(x=11.5, c="red")
    plt.axvline(x=14.5, c="red")
    plt.axvline(x=19.5, c="red")
    plt.axvline(x=22.5, c="red")
    plt.axvline(x=29.5, c="red")
    if horizontal_line is not None:
        plt.axhline(y=horizontal_line, c="red", lw=1, ls="dashed")

    # axes
    format_axes(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        ylim=ylim, 
        min_whiskers_bottom=min_whiskers_bottom, 
        max_whiskers_top=max_whiskers_top, 
        marge=marge)
    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)
    
    # show frame
    plt.show()
    
    return

plot_boxplot_feature("nb_rna", df_30_no_drug, ylim="auto")

In [ ]:
def plot_boxplot_foci(feature, data, 
                      ylim="auto", horizontal_line=None,
                      show_whisker=False, show_flier=False, random_flier=True,
                      path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro", 
                           "ASPM", "ASPM_puro"]
    nuclear_edge = ["SPEN"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["MYH3", "MYH3_puro", "CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro",
                  "RAB13",  "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (p_bodies + translation_factory + random + protrusion + nuclear_edge 
                          + perinuclear + intranuclear)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)
    
    # get whiskers values
    max_whiskers_top, min_whiskers_bottom, marge = get_whiskers_values(data=data, feature=feature)
    
    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # boxplot
    plot_boxplot(
        data=data, 
        feature=feature, 
        genes_distribution=genes_distribution, 
        ax=ax, 
        show_whisker=show_whisker, 
        show_flier=show_flier, 
        random_flier=random_flier)
        
    # text and lines
    plt.axvline(x=6.5, c="red")
    plt.axvline(x=14.5, c="red")
    if horizontal_line is not None:
        plt.axhline(y=horizontal_line, c="red", lw=1, ls="dashed")

    # axes
    format_axes(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        ylim=ylim, 
        min_whiskers_bottom=min_whiskers_bottom, 
        max_whiskers_top=max_whiskers_top, 
        marge=marge)
    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)
        
    # show frame
    plt.show()
    
    return

plot_boxplot_foci("nb_rna", df_30_no_drug, ylim="auto")

In [ ]:
def plot_boxplot_foci_no_puro(feature, data, 
                              ylim="auto", horizontal_line=None,
                              show_whisker=False, show_flier=False, random_flier=True,
                              path_output=None, extension=None):
    # parameters
    p_bodies = ["HMMR", "AURKA", "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "BUB1", "CTNNB1", "ASPM"]
    nuclear_edge = ["SPEN"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    cell_edge = ["FLNA"]
    intranuclear = ["MYH3", "CEP192"]
    protrusion = ["KIF1C", "RAB13", "KIF4A", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

    genes_distribution = (translation_factory + p_bodies + random + protrusion + cell_edge 
                          + nuclear_edge + perinuclear + intranuclear)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)
    
    # get whiskers values
    max_whiskers_top, min_whiskers_bottom, marge = get_whiskers_values(data=data, feature=feature)
    
    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # boxplot
    plot_boxplot(
        data=data, 
        feature=feature, 
        genes_distribution=genes_distribution, 
        ax=ax, 
        show_whisker=show_whisker, 
        show_flier=show_flier, 
        random_flier=random_flier)
        
    # text and lines
    plt.axvline(x=3.5, c="red")
    plt.axvline(x=8.5, c="red")
    if horizontal_line is not None:
        plt.axhline(y=horizontal_line, c="red", lw=1, ls="dashed")

    # axes
    format_axes(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        ylim=ylim, 
        min_whiskers_bottom=min_whiskers_bottom, 
        max_whiskers_top=max_whiskers_top, 
        marge=marge)
    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)
        
    # show frame
    plt.show()
    
    return

plot_boxplot_foci_no_puro("nb_foci", df_30_no_drug, ylim="auto")

In [ ]:
def plot_boxplot_protrusion(feature, data,
                            ylim="auto", horizontal_line=None,
                            show_whisker=False, show_flier=False, random_flier=True,
                            path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro"]
    nuclear_edge = ["SPEN", "ASPM", "ASPM_puro"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro",
                  "MYH3", "MYH3_puro",
                  "RAB13", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (protrusion + random + nuclear_edge + perinuclear 
                          + intranuclear + p_bodies + translation_factory)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)
    
    # get whiskers values
    max_whiskers_top, min_whiskers_bottom, marge = get_whiskers_values(data=data, feature=feature)
    
    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # boxplot
    plot_boxplot(
        data=data, 
        feature=feature, 
        genes_distribution=genes_distribution, 
        ax=ax, 
        show_whisker=show_whisker, 
        show_flier=show_flier, 
        random_flier=random_flier)
        
    # text and lines
    plt.axvline(x=8.5, c="red")
    if horizontal_line is not None:
        plt.axhline(y=horizontal_line, c="red", lw=1, ls="dashed")

    # axes
    format_axes(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        ylim=ylim, 
        min_whiskers_bottom=min_whiskers_bottom, 
        max_whiskers_top=max_whiskers_top, 
        marge=marge)
    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)
        
    # show frame
    plt.show()
    
    return

plot_boxplot_protrusion("nb_rna", df_30_no_drug, ylim="auto")

In [ ]:
def plot_boxplot_nuclear(feature, data,
                         ylim="auto", horizontal_line=None,
                         show_whisker=False, show_flier=False, random_flier=True,
                         path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro"]
    nuclear_edge = ["SPEN", "ASPM", "ASPM_puro"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    cell_edge = ["FLNA"]
    intranuclear = ["MYH3", "MYH3_puro", "CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro",
                  "RAB13", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

    genes_distribution = (perinuclear + nuclear_edge + intranuclear + random 
                          + protrusion + cell_edge + p_bodies + translation_factory)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)
    
    # get whiskers values
    max_whiskers_top, min_whiskers_bottom, marge = get_whiskers_values(data=data, feature=feature)
    
    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # boxplot
    plot_boxplot(
        data=data, 
        feature=feature, 
        genes_distribution=genes_distribution, 
        ax=ax, 
        show_whisker=show_whisker, 
        show_flier=show_flier, 
        random_flier=random_flier)
        
    # text and lines
    plt.axvline(x=4.5, c="red")
    plt.axvline(x=7.5, c="red")
    plt.axvline(x=10.5, c="red")
    if horizontal_line is not None:
        plt.axhline(y=horizontal_line, c="red", lw=1, ls="dashed")

    # axes
    format_axes(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        ylim=ylim, 
        min_whiskers_bottom=min_whiskers_bottom, 
        max_whiskers_top=max_whiskers_top, 
        marge=marge)
    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)
        
    # show frame
    plt.show()
    
    return

plot_boxplot_nuclear("nb_rna", df_30_no_drug, ylim="auto")

In [ ]:
def plot_boxplot_annotations(feature, data,
                             ylim="auto", horizontal_line=None,
                             show_whisker=False, show_flier=False, random_flier=True,
                             path_output=None, extension=None):
    # parameters
    data_annotated = data.loc[~data.loc[:, "label"].isna(), :]
    labels_distribution = ["random", "foci", "intranuclear", "nuclear", "perinuclear", "protrusion"]
    
    # get number of cells
    x_str = []
    for label in labels_distribution:
        nb_cells = int(data_annotated.loc[data_annotated.label == label, ["id_cell", "label"]].groupby("label").count().sum())
        new_str = label + " ({0})".format(nb_cells)
        x_str.append(new_str)
    x_ticks = [i for i in range(len(x_str))]
    
    # get whiskers values
    max_value = data_annotated.loc[:, feature].max()
    min_value = data_annotated.loc[:, feature].min()
    df_whiskers = data_annotated.loc[:, ["label", feature]].groupby(by="label").agg([q1, q3]).stack(level=0)
    df_whiskers["IQR"] = df_whiskers["q3"] - df_whiskers["q1"]
    df_whiskers["whiskers_top"] = df_whiskers["q3"] + df_whiskers["IQR"] * 1.5
    df_whiskers["whiskers_bottom"] = df_whiskers["q1"] - df_whiskers["IQR"] * 1.5
    max_whiskers_top = min(df_whiskers["whiskers_top"].max(), max_value)
    min_whiskers_bottom = max(df_whiskers["whiskers_bottom"].min(), min_value)
    marge = 5 * max_whiskers_top / 100
    
    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # boxplot
    boxprops = dict(linestyle='-', linewidth=2, edgecolor='black')
    flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
    medianprops = dict(linestyle='-', linewidth=2, color='black')
    meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
    capprops = dict(linestyle='-', linewidth=1.5, color='grey')
    whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
    if show_whisker:
        sns.boxplot(x="label", y=feature, data=data_annotated, order=labels_distribution, ax=ax,
                    showmeans=True, meanline=False, meanprops=meanprops,
                    boxprops=boxprops,
                    showfliers=show_flier, flierprops=flierprops, 
                    medianprops=medianprops, 
                    capprops=capprops,
                    whiskerprops=whiskerprops, whis=1.5)
    else:
        sns.boxplot(x="label", y=feature, data=data_annotated, order=labels_distribution, ax=ax,
                    showmeans=True, meanline=False, meanprops=meanprops,
                    boxprops=boxprops,
                    showfliers=show_flier, flierprops=flierprops, 
                    medianprops=medianprops, 
                    showcaps=False, capprops=capprops,
                    whiskerprops=whiskerprops, whis=0)
    
    # scatter plot
    if random_flier:
        data_ = data_annotated.query("label in {0}".format(labels_distribution))
        int_label = {}
        for i, label in enumerate(labels_distribution):
            int_label[label] = i
        values_x = data_.apply(lambda row: int_label[row["label"]], axis=1)
        values_x = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_x
        values_y = data_.loc[:, feature]
        plt.scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.1)
        
    # text and lines
    plt.axvline(x=0.5, c="red")
    plt.axvline(x=1.5, c="red")
    plt.axvline(x=4.5, c="red")
    if horizontal_line is not None:
        plt.axhline(y=horizontal_line, c="red", lw=1, ls="dashed")

    # axes
    labels_distribution = ["random", "foci", "intranuclear", "nuclear edge", "perinuclear", "protrusion"]
    format_axes(
        x_ticks=x_ticks, 
        x_labels=labels_distribution, 
        ylim=ylim, 
        min_whiskers_bottom=min_whiskers_bottom, 
        max_whiskers_top=max_whiskers_top, 
        marge=marge,
        rotation_x=0)
    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)    
    
    # show frame
    plt.show()
    
    return

plot_boxplot_annotations("nb_rna", df_30_no_drug, ylim="auto")

## Specific boxplots

In [ ]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

In [ ]:
def plot_boxplot_feature_pairs_5(feature, data,
                                 xlim="auto", figsize=(15, 10),
                                 show_whisker=False, show_flier=False, random_flier=True,
                                 path_output=None, extension=None):
    # parameters
    keys_distributions = [['AURKA','AURKA_puro'], 
                          ['ASPM', 'ASPM_puro'], ['BUB1', 'BUB1_puro'],
                          ['DYNC1H1', 'DYNC1H1_puro'], ['CTNNB1', 'CTNNB1_puro']]
    coordinates = [0, 1, 2, 3, 4]
    
    if xlim == "auto":
        fig, ax = plt.subplots(5, 1, figsize=(4, 5))
    else:
        fig, ax = plt.subplots(5, 1, figsize=(4, 5), sharex=True)
        
    # get number of cells
    for i, keys_distribution in enumerate(keys_distributions):
        
        gene_ = keys_distribution[0]
        if "_" in gene_:
            gene = gene_.split("_")[0]
        else:
            gene = gene_
        y_labels = []
        for key in keys_distribution:
            if "puro" in key:
                y_labels.append(key.split("_")[0] + "*")
            elif "DMSO" in key or "LG007" in key:
                y_labels.append(" ".join(key.split("_")))
            else:
                y_labels.append(key.split("_")[0])
        y_ticks = [i for i in range(len(y_labels))]

        # get whiskers values
        max_value = data.loc[data.gene == gene, feature].max()
        min_value = data.loc[data.gene == gene, feature].min()
        df_whiskers = data.loc[data.gene == gene, ["key", feature]].groupby(by="key").agg([q1, q3]).stack(level=0)
        df_whiskers["IQR"] = df_whiskers["q3"] - df_whiskers["q1"]
        df_whiskers["whiskers_top"] = df_whiskers["q3"] + df_whiskers["IQR"] * 1.5
        df_whiskers["whiskers_bottom"] = df_whiskers["q1"] - df_whiskers["IQR"] * 1.5
        max_whiskers_top = min(df_whiskers["whiskers_top"].max(), max_value)
        min_whiskers_bottom = max(df_whiskers["whiskers_bottom"].min(), min_value)
        if min_whiskers_bottom == max_whiskers_top: 
            min_whiskers_bottom = min_value
            max_whiskers_top = max_value
        marge = 5 * max_whiskers_top / 100

        # boxplot
        boxprops = dict(linestyle='-', linewidth=4, edgecolor='black')
        flierprops = dict(marker='.', markerfacecolor='gray', markersize=7, markeredgecolor='gray')
        medianprops = dict(linestyle='-', linewidth=4, color='black')
        meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick', markersize=10)
        capprops = dict(linestyle='-', linewidth=3, color='grey')
        whiskerprops = dict(linestyle='-', linewidth=3, color='grey')
        if show_whisker:
            sns.boxplot(x=feature, y="key", data=data, order=keys_distribution, orient="h", ax=ax[i],
                        color="#4daf4a", 
                        showmeans=True, meanline=False, meanprops=meanprops,
                        boxprops=boxprops,
                        showfliers=show_flier, flierprops=flierprops, 
                        medianprops=medianprops, 
                        showcaps=True, capprops=capprops,
                        whiskerprops=whiskerprops, whis=1.5)
        else:
            sns.boxplot(x=feature, y="key", data=data, order=keys_distribution, orient="h", ax=ax[i],
                        color="#4daf4a",
                        showmeans=True, meanline=False, meanprops=meanprops,
                        boxprops=boxprops,
                        showfliers=show_flier, flierprops=flierprops, 
                        medianprops=medianprops, 
                        showcaps=False, capprops=capprops,
                        whiskerprops=whiskerprops, whis=0)
        
        # scatter plot
        if random_flier:
            data_ = data.query("key in {0}".format(keys_distribution))
            values_x = data_.loc[:, feature]
            values_y = data_.loc[:, "key"] != keys_distribution[0]
            values_y = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_y
            ax[i].scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.4)
        
        # axes
        ax[i].set_yticklabels(y_labels, fontweight="bold", fontsize=15)
        ax[i].set_ylabel("")
        ax[i].set_xlabel("")

        if xlim == "auto":
            ax[i].set_xlim(min_whiskers_bottom - marge, max_whiskers_top + marge)
        else:
            ax[i].set_xlim(xlim[0], xlim[1])

    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)    
    
    # show frame
    plt.show()
    
    return

plot_boxplot_feature_pairs_5("nb_foci", df_30, xlim=(0, 12), figsize=(15, 5))

In [ ]:
def plot_boxplot_feature_pairs_6(feature, data,
                                 xlim="auto", figsize=(15, 10),
                                 show_whisker=False, show_flier=False, random_flier=True,
                                 path_output=None, extension=None):
    # parameters
    keys_distributions = [['ASPM', 'ASPM_puro'], ['BUB1', 'BUB1_puro'],
                          ['DYNC1H1', 'DYNC1H1_puro'], ['CTNNB1', 'CTNNB1_puro'],
                          ['AURKA','AURKA_puro'], ['CTNNB1_DMSO', 'CTNNB1_LG007']]
        
    coordinates = [[0, 0], [0, 1], [1, 0], [1, 1], [2, 0], [2, 1]]
    
    if xlim == "auto":
        fig, ax = plt.subplots(3, 2, figsize=(15, 5))
    else:
        fig, ax = plt.subplots(3, 2, figsize=(15, 5), sharex=True)
        
    # get number of cells
    for i, keys_distribution in enumerate(keys_distributions):
        i_row = coordinates[i][0]
        i_col = coordinates[i][1]
        
        gene_ = keys_distribution[0]
        if "_" in gene_:
            gene = gene_.split("_")[0]
        else:
            gene = gene_
        y_labels = []
        for key in keys_distribution:
            if "puro" in key:
                y_labels.append(key.split("_")[0] + "*")
            elif "DMSO" in key or "LG007" in key:
                y_labels.append(" ".join(key.split("_")))
            else:
                y_labels.append(key.split("_")[0])
        y_ticks = [i for i in range(len(y_labels))]

        # get whiskers values
        max_value = data.loc[data.gene == gene, feature].max()
        min_value = data.loc[data.gene == gene, feature].min()
        df_whiskers = data.loc[data.gene == gene, ["key", feature]].groupby(by="key").agg([q1, q3]).stack(level=0)
        df_whiskers["IQR"] = df_whiskers["q3"] - df_whiskers["q1"]
        df_whiskers["whiskers_top"] = df_whiskers["q3"] + df_whiskers["IQR"] * 1.5
        df_whiskers["whiskers_bottom"] = df_whiskers["q1"] - df_whiskers["IQR"] * 1.5
        max_whiskers_top = min(df_whiskers["whiskers_top"].max(), max_value)
        min_whiskers_bottom = max(df_whiskers["whiskers_bottom"].min(), min_value)
        if min_whiskers_bottom == max_whiskers_top: 
            min_whiskers_bottom = min_value
            max_whiskers_top = max_value
        marge = 5 * max_whiskers_top / 100

        # boxplot
        boxprops = dict(linestyle='-', linewidth=4, edgecolor='black')
        flierprops = dict(marker='.', markerfacecolor='gray', markersize=7, markeredgecolor='gray')
        medianprops = dict(linestyle='-', linewidth=4, color='black')
        meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick', markersize=10)
        capprops = dict(linestyle='-', linewidth=3, color='grey')
        whiskerprops = dict(linestyle='-', linewidth=3, color='grey')
        if show_whisker:
            sns.boxplot(x=feature, y="key", data=data, order=keys_distribution, orient="h", ax=ax[i_row, i_col],
                        color="#4daf4a", 
                        showmeans=True, meanline=False, meanprops=meanprops,
                        boxprops=boxprops,
                        showfliers=show_flier, flierprops=flierprops, 
                        medianprops=medianprops, 
                        showcaps=True, capprops=capprops,
                        whiskerprops=whiskerprops, whis=1.5)
        else:
            sns.boxplot(x=feature, y="key", data=data, order=keys_distribution, orient="h", ax=ax[i_row, i_col],
                        color="#4daf4a",
                        showmeans=True, meanline=False, meanprops=meanprops,
                        boxprops=boxprops,
                        showfliers=show_flier, flierprops=flierprops, 
                        medianprops=medianprops, 
                        showcaps=False, capprops=capprops,
                        whiskerprops=whiskerprops, whis=0)
        
        # scatter plot
        if random_flier:
            data_ = data.query("key in {0}".format(keys_distribution))
            values_x = data_.loc[:, feature]
            values_y = data_.loc[:, "key"] != keys_distribution[0]
            values_y = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_y
            ax[i_row, i_col].scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.4)
        
        # axes
        ax[i_row, i_col].set_yticklabels(y_labels, fontweight="bold", fontsize=15)
        ax[i_row, i_col].set_ylabel("")
        ax[i_row, i_col].set_xlabel("")

        if xlim == "auto":
            ax[i_row, i_col].set_xlim(min_whiskers_bottom - marge, max_whiskers_top + marge)
        else:
            ax[i_row, i_col].set_xlim(xlim[0], xlim[1])

    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)    
    
    # show frame
    plt.show()
    
    return

plot_boxplot_feature_pairs_6("nb_foci", df_30, xlim=(0, 12), figsize=(15, 5))

In [ ]:
def plot_boxplot_feature_pairs_all(feature, data,
                                   xlim="auto", figsize=(15, 10),
                                   show_whisker=False, show_flier=False, random_flier=True,
                                   path_output=None, extension=None):
    # parameters
    keys_distributions = [['ASPM', 'ASPM_puro'], ['BUB1', 'BUB1_puro'],
                          ['DYNC1H1', 'DYNC1H1_puro'], ['CTNNB1', 'CTNNB1_puro'],
                          ['AURKA','AURKA_puro'], ['CTNNB1_DMSO', 'CTNNB1_LG007'],
                          ['HMMR', 'HMMR_puro'], ['MYH3', 'MYH3_puro'],
                          ['KIF1C', 'KIF1C_puro'], ['KIF4A', 'KIF4A_puro']]
        
    coordinates = [[0, 0], [0, 1], [1, 0], [1, 1], [2, 0], [2, 1], [3, 0], [3, 1], [4, 0], [4, 1]]

    if xlim == "auto":
        fig, ax = plt.subplots(5, 2, figsize=figsize)
    else:
        fig, ax = plt.subplots(5, 2, figsize=figsize, sharex=True)
        
    # get number of cells
    for i, keys_distribution in enumerate(keys_distributions):
        i_row = coordinates[i][0]
        i_col = coordinates[i][1]
        
        gene_ = keys_distribution[0]
        if "_" in gene_:
            gene = gene_.split("_")[0]
        else:
            gene = gene_
        y_labels = []
        for key in keys_distribution:
            if "puro" in key:
                y_labels.append(key.split("_")[0] + "*")
            elif "DMSO" in key or "LG007" in key:
                y_labels.append(" ".join(key.split("_")))
            else:
                y_labels.append(key.split("_")[0])
        y_ticks = [i for i in range(len(y_labels))]

        # get whiskers values
        max_value = data.loc[data.gene == gene, feature].max()
        min_value = data.loc[data.gene == gene, feature].min()
        df_whiskers = data.loc[data.gene == gene, ["key", feature]].groupby(by="key").agg([q1, q3]).stack(level=0)
        df_whiskers["IQR"] = df_whiskers["q3"] - df_whiskers["q1"]
        df_whiskers["whiskers_top"] = df_whiskers["q3"] + df_whiskers["IQR"] * 1.5
        df_whiskers["whiskers_bottom"] = df_whiskers["q1"] - df_whiskers["IQR"] * 1.5
        max_whiskers_top = min(df_whiskers["whiskers_top"].max(), max_value)
        min_whiskers_bottom = max(df_whiskers["whiskers_bottom"].min(), min_value)
        if min_whiskers_bottom == max_whiskers_top: 
            min_whiskers_bottom = min_value
            max_whiskers_top = max_value
        marge = 5 * max_whiskers_top / 100

        # boxplot
        boxprops = dict(linestyle='-', linewidth=4, edgecolor='black')
        flierprops = dict(marker='.', markerfacecolor='gray', markersize=7, markeredgecolor='gray')
        medianprops = dict(linestyle='-', linewidth=4, color='black')
        meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick', markersize=10)
        capprops = dict(linestyle='-', linewidth=3, color='grey')
        whiskerprops = dict(linestyle='-', linewidth=3, color='grey')
        if show_whisker:
            sns.boxplot(x=feature, y="key", data=data, order=keys_distribution, orient="h", ax=ax[i_row, i_col],
                        color="#4daf4a", 
                        showmeans=True, meanline=False, meanprops=meanprops,
                        boxprops=boxprops,
                        showfliers=show_flier, flierprops=flierprops, 
                        medianprops=medianprops, 
                        showcaps=True, capprops=capprops,
                        whiskerprops=whiskerprops, whis=1.5)
        else:
            sns.boxplot(x=feature, y="key", data=data, order=keys_distribution, orient="h", ax=ax[i_row, i_col],
                        color="#4daf4a",
                        showmeans=True, meanline=False, meanprops=meanprops,
                        boxprops=boxprops,
                        showfliers=show_flier, flierprops=flierprops, 
                        medianprops=medianprops, 
                        showcaps=False, capprops=capprops,
                        whiskerprops=whiskerprops, whis=0)
        
        # scatter plot
        if random_flier:
            data_ = data.query("key in {0}".format(keys_distribution))
            values_x = data_.loc[:, feature]
            values_y = data_.loc[:, "key"] != keys_distribution[0]
            values_y = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_y
            ax[i_row, i_col].scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.4)
        
        # axes
        ax[i_row, i_col].set_yticklabels(y_labels, fontweight="bold", fontsize=15)
        ax[i_row, i_col].set_ylabel("")
        ax[i_row, i_col].set_xlabel("")

        if xlim == "auto":
            ax[i_row, i_col].set_xlim(min_whiskers_bottom - marge, max_whiskers_top + marge)
        else:
            ax[i_row, i_col].set_xlim(xlim[0], xlim[1])

    plt.tight_layout()
    
    # save frame
    save_frame(
        path_output=path_output, 
        extension=extension)    
    
    # show frame
    plt.show()
    
    return

plot_boxplot_feature_pairs_all("nb_foci", df_30, xlim=(0, 12), figsize=(15, 7))

## Barplots

In [ ]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

def get_nb_cells(data, genes_distribution):
    # get number of cells
    x_labels = []
    for key in genes_distribution:
        nb_cells = int(data.loc[data.key == key, ["id_cell", "key"]].groupby("key").count().sum())
        if "puro" in key:
            key = key.split("_")[0] + "*"
        new_label = key
        x_labels.append(new_label)
    x_ticks = [i for i in range(len(x_labels))]
    
    return x_ticks, x_labels

def format_axes_2(x_ticks, x_labels, rotation_x=90):
    # axes
    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=rotation_x, fontweight="bold", fontsize=15)
    plt.yticks(rotation=0, fontweight="bold", fontsize=15)
    plt.xlabel("")
    plt.ylabel("")
    plt.ylim(0, 1)
        
    return

def save_frame(path_output, extension):
    # save frame
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
    
    return

In [ ]:
def plot_barplot_general(feature, data, path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro"]
    nuclear_edge = ["SPEN", "ASPM", "ASPM_puro"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["MYH3", "MYH3_puro", "CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro", 
                  "RAB13", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (random + protrusion + nuclear_edge + perinuclear 
                          + intranuclear + p_bodies + translation_factory)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)
    
    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # barplot
    sns.barplot(x="key", y=feature, data=data, 
                color="#4daf4a", order=genes_distribution, 
                estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
                errcolor='.26', errwidth=2, capsize=0.2, dodge=True, ax=ax)

    # text and lines
    plt.axvline(x=4.5, c="red")
    plt.axvline(x=11.5, c="red")
    plt.axvline(x=14.5, c="red")
    plt.axvline(x=19.5, c="red")
    plt.axvline(x=22.5, c="red")
    plt.axvline(x=29.5, c="red")

    # axes
    format_axes_2(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        rotation_x=90)
    plt.tight_layout()
    
    # save frame
    save_frame(path_output, extension)
    
    # show frame
    plt.show()
    
    return

plot_barplot_general("nb_rna", df_30_no_drug)

In [ ]:
def plot_barplot_foci(feature, data, path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro", 
                           "ASPM", "ASPM_puro"]
    nuclear_edge = ["SPEN"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["MYH3", "MYH3_puro", "CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro",
                  "RAB13", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (p_bodies + translation_factory + random + protrusion + 
                          nuclear_edge + perinuclear + intranuclear)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)
    
    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # barplot
    sns.barplot(x="key", y=feature, data=data, 
                color="#4daf4a", order=genes_distribution, 
                estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
                errcolor='.26', errwidth=2, capsize=0.2, dodge=True, ax=ax)

    # text and lines
    plt.axvline(x=6.5, c="red")
    plt.axvline(x=14.5, c="red")

    # axes
    format_axes_2(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        rotation_x=90)
    plt.tight_layout()
    
    # save frame
    save_frame(path_output, extension)
    
    # show frame
    plt.show()
    
    return

plot_barplot_foci("nb_rna", df_30_no_drug)

In [ ]:
def plot_barplot_foci_no_puro(feature, data, path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "HMMR", "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "BUB1", "CTNNB1", "ASPM"]
    nuclear_edge = ["SPEN"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["MYH3", "CEP192"]
    protrusion = ["KIF1C", "RAB13", "KIF4A", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (p_bodies + translation_factory + random + protrusion 
                          + nuclear_edge + perinuclear + intranuclear)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)

    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # barplot
    sns.barplot(x="key", y=feature, data=data, 
                color="#4daf4a", order=genes_distribution, 
                estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
                errcolor='.26', errwidth=2, capsize=0.2, dodge=True, ax=ax)

    # text and lines
    plt.axvline(x=4.5, c="red")
    plt.axvline(x=8.5, c="red")

    # axes
    format_axes_2(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        rotation_x=90)
    plt.tight_layout()
    
    # save frame
    save_frame(path_output, extension)
    
    # show frame
    plt.show()
    
    return

plot_barplot_foci_no_puro("nb_rna", df_30_no_drug)

In [ ]:
def plot_barplot_nuclear(feature, data, path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro"]
    nuclear_edge = ["SPEN", "ASPM", "ASPM_puro"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["MYH3", "MYH3_puro", "CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro", 
                  "RAB13", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (perinuclear + nuclear_edge + intranuclear + random + 
                          protrusion + p_bodies + translation_factory)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)

    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # barplot
    sns.barplot(x="key", y=feature, data=data, 
                color="#4daf4a", order=genes_distribution, 
                estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
                errcolor='.26', errwidth=2, capsize=0.2, dodge=True, ax=ax)

    # text and lines
    plt.axvline(x=4.5, c="red")
    plt.axvline(x=7.5, c="red")
    plt.axvline(x=10.5, c="red")

    # axes
    format_axes_2(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        rotation_x=90)
    plt.tight_layout()
    
    # save frame
    save_frame(path_output, extension)
    
    # show frame
    plt.show()
    
    return

plot_barplot_nuclear("nb_rna", df_30_no_drug)

In [ ]:
def plot_barplot_protrusion(feature, data, path_output=None, extension=None):
    # parameters
    p_bodies = ["AURKA", "AURKA_puro", 
                "HMMR", "HMMR_puro", 
                "CEP170P1", "CRKL", "PAK2"]
    translation_factory = ["DYNC1H1", "DYNC1H1_puro", 
                           "BUB1", "BUB1_puro", 
                           "CTNNB1", "CTNNB1_puro"]
    nuclear_edge = ["SPEN", "ASPM", "ASPM_puro"]
    perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
    intranuclear = ["CEP192"]
    protrusion = ["KIF1C", "KIF1C_puro", 
                  "KIF4A", "KIF4A_puro", 
                  "MYH3", "MYH3_puro",
                  "RAB13", "KIF5B", "DYNLL2"]
    random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC", "FLNA"]

    genes_distribution = (protrusion + random + nuclear_edge + perinuclear + 
                          intranuclear + p_bodies + translation_factory)

    # get number of cells
    x_ticks, x_labels = get_nb_cells(data=data, genes_distribution=genes_distribution)

    # plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # barplot
    sns.barplot(x="key", y=feature, data=data, 
                color="#4daf4a", order=genes_distribution, 
                estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
                errcolor='.26', errwidth=2, capsize=0.2, dodge=True, ax=ax)

    # text and lines
    plt.axvline(x=8.5, c="red")

    # axes
    format_axes_2(
        x_ticks=x_ticks, 
        x_labels=x_labels, 
        rotation_x=90)
    plt.tight_layout()
    
    # save frame
    save_frame(path_output, extension)
    
    # show frame
    plt.show()
    
    return

plot_barplot_protrusion("nb_rna", df_30_no_drug)

# Distribution plots

## Foci

In [ ]:
features = ["nb_foci", 
            "proportion_rna_in_foci", 
            "index_foci_mean_distance_cyt", 
            "index_foci_median_distance_cyt", 
            "index_foci_mean_distance_nuc", 
            "index_foci_median_distance_nuc"]

In [ ]:
for feat in features:
    print(feat)
    plot_boxplot_annotations(feature=feat, data=df_30_no_drug,
                             ylim="auto", show_whisker=False, show_flier=False, 
                             random_flier=True, horizontal_line=None)
    plot_boxplot_foci(feature=feat, data=df_30_no_drug,
                      ylim="auto", show_whisker=False, show_flier=False, 
                      random_flier=True, horizontal_line=None)

In [ ]:
plot_boxplot_foci(feature="nb_foci", data=df_30_no_drug, 
                  ylim="auto", show_whisker=False, show_flier=False, 
                  random_flier=True, horizontal_line=None,
                  path_output=os.path.join(plot_directory, "boxplot_nb_foci"), 
                  extension=["png", "pdf"])
plot_boxplot_foci(feature="proportion_rna_in_foci", data=df_30_no_drug, 
                  ylim=(0, 1), show_whisker=False, show_flier=False, 
                  random_flier=True, horizontal_line=None,
                  path_output=os.path.join(plot_directory, "boxplot_proportion_rna_in_foci"), 
                  extension=["png", "pdf"])
plot_boxplot_foci_no_puro(feature="proportion_rna_in_foci", data=df_30_no_drug, 
                          ylim=(0, 0.5), show_whisker=False, show_flier=False, 
                          random_flier=True, horizontal_line=None,
                          path_output=os.path.join(plot_directory, "boxplot_proportion_rna_in_foci_no_puro"), 
                          extension=["png", "pdf"])

## Protrusion

In [ ]:
features  = ["index_rna_opening_30", 
             "score_polarization_cyt", 
             "score_polarization_nuc", 
             "index_dispersion", 
             "index_peripheral_dispersion"]

In [ ]:
for feat in features:
    print(feat)
    plot_boxplot_annotations(feature=feat, data=df_30_no_drug,
                             ylim="auto", show_whisker=False, show_flier=False, 
                             random_flier=True, horizontal_line=None)
    plot_boxplot_protrusion(feature=feat, data=df_30_no_drug,
                            ylim="auto", show_whisker=False, show_flier=False, 
                            random_flier=True, horizontal_line=None)

In [ ]:
plot_boxplot_protrusion(feature="index_rna_opening_30", data=df_30_no_drug, 
                        ylim="auto", show_whisker=False, show_flier=False, 
                        random_flier=True, horizontal_line=1.,
                        path_output=os.path.join(plot_directory, "boxplot_index_rna_opening_30"), 
                        extension=["png", "pdf"])
plot_boxplot_protrusion(feature="index_peripheral_dispersion", data=df_30_no_drug, 
                        ylim="auto", show_whisker=False, show_flier=False, 
                        random_flier=True, horizontal_line=1.,
                        path_output=os.path.join(plot_directory, "boxplot_index_peripheral_dispersion"), 
                        extension=["png", "pdf"])
plot_boxplot_protrusion(feature="proportion_rna_opening_30", data=df_30_no_drug, 
                        ylim=(0, 1), show_whisker=False, show_flier=False, 
                        random_flier=True, horizontal_line=None, 
                        path_output=os.path.join(plot_directory, "boxplot_proportion_rna_opening_30"), 
                        extension=["png", "pdf"])

### Specific plots for the paper

In [ ]:
path_output = os.path.join(plot_directory, "plot_s2a_boxplot_proportion_rna_opening_30")
extension = extension=["png", "pdf"]
feature = "proportion_rna_opening_30"

# parameters
protrusion = ["KIF1C", "KIF4A", "MYH3", "RAB13", "KIF5B", "DYNLL2"]
other = ["KIF20B", "MYO18A", "DYNC1H1"]
genes_distribution = protrusion + other

# plot
fig, ax = plt.subplots(figsize=(15, 5))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black', alpha=0.95)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="key", y=feature, data=df_30_no_drug, order=genes_distribution, ax=ax,
            color="#4daf4a",
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0,
            orient="v")

# scatter plot
data_ = df_30_no_drug.query("key in {0}".format(genes_distribution))
int_gene = {}
for i, gene in enumerate(genes_distribution):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.2)

# text and lines
plt.axvline(x=5.5, c="red")

# axes
plt.xlabel("")
plt.ylabel("")
plt.yticks(fontweight="bold", fontsize=15)
plt.xticks(fontweight="bold", fontsize=15)
plt.ylim(0, 0.6)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_s2a_boxplot_index_rna_opening_30")
extension = extension=["png", "pdf"]
feature = "index_rna_opening_30"

# parameters
protrusion = ["KIF1C", "KIF4A", "MYH3", "RAB13", "KIF5B", "DYNLL2"]
other = ["KIF20B", "MYO18A", "DYNC1H1"]
genes_distribution = protrusion + other

# plot
fig, ax = plt.subplots(figsize=(15, 5))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black', alpha=0.95)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="key", y=feature, data=df_30_no_drug, order=genes_distribution, ax=ax,
            color="#4daf4a",
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0,
            orient="v")

# scatter plot
data_ = df_30_no_drug.query("key in {0}".format(genes_distribution))
int_gene = {}
for i, gene in enumerate(genes_distribution):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.2)

# text and lines
plt.axvline(x=5.5, c="red")
plt.axhline(y=1., c="red", lw=1, ls="dashed")

# axes
plt.xlabel("")
plt.ylabel("")
plt.yticks(fontweight="bold", fontsize=15)
plt.xticks(fontweight="bold", fontsize=15)
plt.ylim(0, 6)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

## Nuclear

In [ ]:
features = ["index_mean_distance_cyt", 
            "index_median_distance_cyt", 
            "index_mean_distance_nuc", 
            "index_median_distance_nuc", 
            "proportion_rna_in_nuc"]

In [ ]:
for feat in features:
    print(feat)
    plot_boxplot_annotations(feature=feat, data=df_30_no_drug,
                             ylim="auto", show_whisker=False, show_flier=False, 
                             random_flier=True, horizontal_line=None)
    plot_boxplot_nuclear(feature=feat, data=df_30_no_drug,
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=None)

In [ ]:
plot_boxplot_nuclear(feature="proportion_rna_in_nuc", data=df_30_no_drug, 
                     ylim=(0, 1), show_whisker=False, show_flier=False, 
                     random_flier=True, horizontal_line=None,
                     path_output=os.path.join(plot_directory, "boxplot_proportion_rna_in_nuc"), 
                     extension=["png", "pdf"])
plot_boxplot_nuclear(feature="index_mean_distance_cyt", data=df_30_no_drug, 
                     ylim="auto", show_whisker=False, show_flier=False, 
                     random_flier=True, horizontal_line=1.,
                     path_output=os.path.join(plot_directory, "boxplot_index_mean_distance_cyt"), 
                     extension=["png", "pdf"])
plot_boxplot_nuclear(feature="index_mean_distance_nuc", data=df_30_no_drug, 
                     ylim="auto", show_whisker=False, show_flier=False, 
                     random_flier=True, horizontal_line=1.,
                     path_output=os.path.join(plot_directory, "boxplot_index_mean_distance_nuc"), 
                     extension=["png", "pdf"])

## General

In [ ]:
plot_boxplot_feature("nb_rna", data=df_30_no_drug, 
                     ylim="auto", show_whisker=False, show_flier=False, 
                     random_flier=True, horizontal_line=None,
                     path_output=os.path.join(plot_directory, "boxplot_nb_rna"), 
                     extension=["png", "pdf"])
plot_boxplot_feature("proportion_nuc_area", data=df_30_no_drug, 
                     ylim=(0, 1), show_whisker=False, show_flier=False, 
                     random_flier=True, horizontal_line=None,
                     path_output=os.path.join(plot_directory, "boxplot_proportion_nuc_area"), 
                     extension=["png", "pdf"])
plot_boxplot_feature(feature="index_foci_mean_distance_cyt", data=df_30_no_drug, 
                     ylim="auto", show_whisker=False, show_flier=False, 
                     random_flier=True, horizontal_line=1.,
                     path_output=os.path.join(plot_directory, "boxplot_index_foci_mean_distance_cyt"), 
                     extension=["png", "pdf"])
plot_boxplot_feature(feature="index_foci_mean_distance_nuc", data=df_30_no_drug, 
                     ylim="auto", show_whisker=False, show_flier=False, 
                     random_flier=True, horizontal_line=1., 
                     path_output=os.path.join(plot_directory, "boxplot_index_foci_mean_distance_nuc"), 
                     extension=["png", "pdf"])

## Annotations

In [ ]:
plot_boxplot_annotations(feature="nb_foci", data=df_30_no_drug, 
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=None,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_nb_foci"), 
                         extension=["png", "pdf"])
plot_boxplot_annotations(feature="proportion_rna_in_foci", data=df_30_no_drug, 
                         ylim=(0, 1), show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=None,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_proportion_rna_in_foci"), 
                         extension=["png", "pdf"])

plot_boxplot_annotations(feature="index_rna_opening_30", data=df_30_no_drug, 
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=1.,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_index_rna_opening_30"), 
                         extension=["png", "pdf"])
plot_boxplot_annotations(feature="index_peripheral_dispersion", data=df_30_no_drug, 
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=1.,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_index_peripheral_dispersion"), 
                         extension=["png", "pdf"])

plot_boxplot_annotations(feature="proportion_rna_in_nuc", data=df_30_no_drug, 
                         ylim=(0, 1), show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=None,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_proportion_rna_in_nuc"), 
                         extension=["png", "pdf"])
plot_boxplot_annotations(feature="index_mean_distance_cyt", data=df_30_no_drug, 
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=1.,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_index_mean_distance_cyt"), 
                         extension=["png", "pdf"])
plot_boxplot_annotations(feature="index_mean_distance_nuc", data=df_30_no_drug, 
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=1.,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_index_mean_distance_nuc"), 
                         extension=["png", "pdf"])

plot_boxplot_annotations(feature="index_foci_mean_distance_cyt", data=df_30_no_drug, 
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=1.,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_index_foci_mean_distance_cyt"), 
                         extension=["png", "pdf"])
plot_boxplot_annotations(feature="index_foci_mean_distance_nuc", data=df_30_no_drug, 
                         ylim="auto", show_whisker=False, show_flier=False, 
                         random_flier=True, horizontal_line=1.,
                         path_output=os.path.join(plot_directory, "boxplot_annotation_index_foci_mean_distance_nuc"), 
                         extension=["png", "pdf"])

## Puromycin and drugs

In [ ]:
path_output = os.path.join(plot_directory, "plot_s8c_boxplot_rna_puro")
extension = ["png", "pdf"]
feature = "nb_rna"

# parameters
genes = ["MYH3",
         "AP1S2", "AKAP9", "AKAP1", "HSP90B1",
         "ASPM",
         "DYNC1H1", "BUB1", "CTNNB1", 
         "HMMR", "AURKA",
         "KIF1C", "KIF4A"]
genes_puro = ["MYH3_puro",
              "AP1S2_puro", "AKAP9_puro", "AKAP1_puro", "HSP90B1_puro",
              "ASPM_puro",
              "DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", 
              "HMMR_puro", "AURKA_puro",
              "KIF1C_puro", "KIF4A_puro"]

# plot
fig, ax = plt.subplots(figsize=(15, 5))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black', alpha=0.95)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="gene", y=feature, data=df_30_no_drug, hue="puromycin",
            palette=["#4381de", "#e83333"], order=genes,
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0, ax=ax,)

# scatter plot no puro
data_ = df_30_no_drug.query("key in {0}".format(genes))
int_gene = {}
for i, gene in enumerate(genes):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=-0.06, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.15)

# scatter plot puro
data_ = df_30_no_drug.query("key in {0}".format(genes_puro))
int_gene = {}
for i, gene in enumerate(genes_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=0.06, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.15)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 2000)
patch_nopuro = mpatches.Patch(color="#4381de", label='No puromycin', alpha=0.95)
patch_puro = mpatches.Patch(color="#e83333", label='Puromycin', alpha=0.95)
plt.legend(handles=[patch_nopuro, patch_puro], loc='upper center', fontsize=15)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_s8c_boxplot_rna_nopuro")
extension = ["png", "pdf"]
feature = "nb_rna"

# parameters
p_bodies = ["HMMR", "AURKA", "CEP170P1", "CRKL", "PAK2"]
translation_factory = ["DYNC1H1", "BUB1", "CTNNB1", "ASPM"]
nuclear_edge = ["SPEN"]
perinuclear = ["ATP6A2", "AP1S2", "AKAP9", "HSP90B1", "AKAP1"]
cell_edge = ["FLNA"]
intranuclear = ["MYH3", "CEP192"]
protrusion = ["KIF1C", "RAB13", "KIF4A", "KIF5B", "DYNLL2"]
random = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

genes = (translation_factory + p_bodies + random + protrusion + cell_edge
         + nuclear_edge + perinuclear + intranuclear)

# plot
fig, ax = plt.subplots(figsize=(15, 5))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black', alpha=0.95)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="gene", y=feature, data=df_30_no_drug,
            color="#4381de", order=genes,
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0, ax=ax,)

# scatter plot no puro
data_ = df_30_no_drug.query("key in {0}".format(genes))
int_gene = {}
for i, gene in enumerate(genes):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.15)

# text and lines
plt.axvline(x=3.5, c="red")
plt.axvline(x=8.5, c="red")

# axes
plt.xticks(rotation=90, fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 2000)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
print("nb_foci")
plot_boxplot_feature_pairs_6("nb_foci", df_30, 
                             xlim=(0, 12), figsize=(15, 5), show_whisker=False, 
                             show_flier=False, random_flier=True,
                             path_output=os.path.join(plot_directory, "boxplot_drugs_nb_foci"), 
                             extension=["png", "pdf"])
print("proportion_rna_in_foci")
plot_boxplot_feature_pairs_6("proportion_rna_in_foci", df_30, 
                             xlim=(0, 1), figsize=(15, 5), show_whisker=False, 
                             show_flier=False, random_flier=True,
                             path_output=os.path.join(plot_directory, "boxplot_drugs_proportion_rna_in_foci"), 
                             extension=["png", "pdf"])
print("nb_rna")
plot_boxplot_feature_pairs_all("nb_rna", df_30, 
                               xlim=(0, 2000), figsize=(15, 7), show_whisker=False, 
                               show_flier=False, random_flier=True,
                               path_output=os.path.join(plot_directory, "boxplot_drugs_nb_rna"), 
                               extension=["png", "pdf"])
print("proportion_nuc_area")
plot_boxplot_feature_pairs_all("proportion_nuc_area", df_30, 
                               xlim=(0, 1), figsize=(15, 7), show_whisker=False, 
                               show_flier=False, random_flier=True,
                               path_output=os.path.join(plot_directory, "boxplot_drugs_proportion_nuc_area"), 
                               extension=["png", "pdf"])
print("proportion_rna_in_foci")
plot_boxplot_feature_pairs_5("proportion_rna_in_foci", df_30, 
                             xlim=(0, 1), figsize=(4, 5), show_whisker=False, 
                             show_flier=False, random_flier=True,
                             path_output=os.path.join(plot_directory, "plot_s8b_boxplot_proportion_rna_in_foci"), 
                             extension=["png", "pdf"])
print("nb_foci")
plot_boxplot_feature_pairs_5("nb_foci", df_30, 
                             xlim=(0, 12), figsize=(4, 5), show_whisker=False, 
                             show_flier=False, random_flier=True,
                             path_output=os.path.join(plot_directory, "plot_s8b_boxplot_nb_foci"), 
                             extension=["png", "pdf"])

## Distribution topography

In [ ]:
path_output = os.path.join(plot_directory, "plot_topography")
extension = ["png", "pdf"]
patterns = ['intranuclear', 'nuclear', 'perinuclear', "protrusion", 'foci', "random"]
colors_pattern = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#a65628", "#f781bf"]
features_nuc = ["proportion_rna_nuc_edge", 
                "proportion_rna_nuc_radius_5_10", 
                "proportion_rna_nuc_radius_10_15", 
                "proportion_rna_nuc_radius_15_20"]
features_cyt = ["proportion_rna_cyt_radius_0_5", 
                "proportion_rna_cyt_radius_5_10", 
                "proportion_rna_cyt_radius_10_15", 
                "proportion_rna_cyt_radius_15_20"]
features_name_nuc = ["-5 - 5", "5 - 10", "10 - 15", "15 - 20"]
features_name_cyt = ["20 - 15", "15 - 10", "10 - 5", "5 - 0"]

# plot
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=False, sharey=True)

# get data
nb_cells_pattern = []
for i, pattern in enumerate(patterns):
    df_topography = df_30_no_drug.loc[df_30_no_drug.loc[:, "annotated"], :]
    df_topography = df_topography.loc[df_topography.loc[:, "exclusive_label"], :]
    df_topography = df_topography.loc[df_topography.loc[:, "label"] == pattern, :]
    nb_cells = df_topography.shape[0]
    nb_cells_pattern.append(nb_cells)
    
    # related to nucleus...
    heights_nuc = []
    x_nuc = []
    ci_top_nuc = []
    ci_down_nuc = []
    for j, feat in enumerate(features_nuc):
        height = df_topography.loc[:, feat].mean()
        heights_nuc.append(height)
        x_nuc.append(j)
        se = sem(df_topography.loc[:, feat])
        ci_range = se * t.ppf((1 + 0.95) / 2., nb_cells-1)
        ci_top_nuc.append(height + ci_range)
        ci_down_nuc.append(height - ci_range)
        
    # plot nuc    
    axes[0].plot(x_nuc, heights_nuc, color=colors_pattern[i], alpha=1, 
                 linewidth=2, linestyle='-', markersize=10, marker="D")
    axes[0].fill_between(x_nuc, ci_top_nuc, ci_down_nuc, color=colors_pattern[i], alpha=0.5)
    axes[0].grid(which='major', axis='y', color="gray", alpha=0.6)
    axes[0].set_ylabel("Average proportion of mRNAs", fontweight="bold", fontsize=15)
    axes[0].set_xlim(-0.5, x_nuc[-1] + 0.25)
    axes[0].set_xlabel("Distance from nucleus membrane (in pixels)", fontweight="bold", fontsize=15)
        
    # ... and cytoplasmic membrane
    heights_cyt = []
    x_cyt = []
    ci_top_cyt = []
    ci_down_cyt = []
    for j, feat in enumerate(features_cyt):
        height = df_topography.loc[:, feat].mean()
        heights_cyt.append(height)
        x_cyt.append(j)
        se = sem(df_topography.loc[:, feat])
        ci_range = se * t.ppf((1 + 0.95) / 2., nb_cells-1)
        ci_top_cyt.append(height + ci_range)
        ci_down_cyt.append(height - ci_range)
    
    # plot cytoplasm
    axes[1].plot(x_cyt, heights_cyt, color=colors_pattern[i], alpha=1, 
                 linewidth=2, linestyle='-', markersize=10, marker="D")
    axes[1].fill_between(x_cyt, ci_top_cyt, ci_down_cyt, color=colors_pattern[i], alpha=0.5)
    axes[1].grid(which='major', axis='y', color="gray", alpha=0.6)
    axes[1].set_xlim(-0.25, x_cyt[-1] + 0.5)
    axes[1].set_xlabel("Distance from cytoplasmic membrane (in pixels)", fontweight="bold", fontsize=15)
    
# Set the ticks and ticklabels
plt.sca(axes[0])
plt.xticks(x_nuc, features_name_nuc,  fontweight="bold", fontsize=13)
plt.yticks(fontweight="bold", fontsize=13)
plt.sca(axes[1])
plt.xticks(x_cyt, features_name_cyt,  fontweight="bold", fontsize=13)

# add a big axis, hide frame
fig.add_subplot(111, frameon=False)

# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)

# texts and limits
plt.text(x=0.02, y=0.07, s="Nucleus", rotation=90,
           color="#de2d26", fontweight="bold", fontsize=20)
plt.text(x=0.951, y=0.05, s="Cytoplasmic \n membrane", rotation=90,
           color="#de2d26", fontweight="bold", fontsize=20)
plt.ylim((0, 0.2))

# legend
legend_elements = []
patterns_names = ['intranuclear', 'nuclear edge', 'perinuclear', "protrusion", 'foci', "random"]
for i in range(len(patterns_names)):
    element = Line2D([0], [0], label="{0} ({1} cells)".format(patterns_names[i], nb_cells_pattern[i]),
                     color=colors_pattern[i], alpha=1, 
                     linewidth=2, linestyle='-',
                     marker="D", markerfacecolor=colors_pattern[i], markersize=10)
    legend_elements.append(element)
plt.legend(handles=legend_elements, prop={'size': 15}, loc='upper center', framealpha=1)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame            
plt.show()

# Welch t-tests

## Function

In [ ]:
def welch_t_test(bigger_key, smaller_key, feature, data, alpha=0.002, verbose=True, two_tail=False):
    bigger_values = data.loc[data.loc[:, "key"] == bigger_key, feature]
    smaller_values = data.loc[data.loc[:, "key"] == smaller_key, feature]
    t, p = ttest_ind(bigger_values, smaller_values, equal_var=False, nan_policy='raise')
    
    if two_tail:
    
        if verbose:
            print("Feature: {0}".format(feature))
            print("{0}: mean {1:.3f} | std {2:.3f}".format(bigger_key, np.mean(bigger_values), np.std(bigger_values)))
            print("{0}: mean {1:.3f} | std {2:.3f}".format(smaller_key, np.mean(smaller_values), np.std(smaller_values)))
            print("\tH0: {0} == {1}".format(bigger_key, smaller_key))
            print("\tH1: {0} != {1}".format(bigger_key, smaller_key))

        if p < alpha:
            print("\t=> H0 rejected at the significant level {0:.4f} (t statistic {1:.3f} | p-value {2:.4f})".format(alpha, t, p))
        else:
            print("\t=> H0 not rejected at the significant level {0:.4f} (t statistic {1:.3f} | p-value {2:.4f})".format(alpha, t, p))
    
    else:
        p /= 2
        alpha /= 2

        if verbose:
            print("Feature: {0}".format(feature))
            print("{0}: mean {1:.3f} | std {2:.3f}".format(bigger_key, np.mean(bigger_values), np.std(bigger_values)))
            print("{0}: mean {1:.3f} | std {2:.3f}".format(smaller_key, np.mean(smaller_values), np.std(smaller_values)))
            print("\tH0: {0} <= {1}".format(bigger_key, smaller_key))
            print("\tH1: {0} > {1}".format(bigger_key, smaller_key))

        if p < alpha:
            if t > 0:
                print("\t=> H0 rejected at the significant level {0:.4f} (t statistic {1:.3f} | p-value {2:.4f})".format(alpha, t, p))
            else:
                raise ValueError("t statistic: {0:.3f}".format(t))
        else:
            if t > 0:
                print("\t=> H0 not rejected at the significant level {0:.4f} (t statistic {1:.3f} | p-value {2:.4f})".format(alpha, t, p))
            else:
                raise ValueError("t statistic: {0:.3f}".format(t))

In [ ]:
welch_t_test(bigger_key="ASPM", smaller_key="ASPM_puro", feature="nb_foci", data=df_30)
print()
welch_t_test(bigger_key="ASPM", smaller_key="ASPM_puro", feature="nb_foci", data=df_30, two_tail=True)

## Number of foci

In [ ]:
keys_distributions = [['ASPM', 'ASPM_puro', False], 
                      ['BUB1', 'BUB1_puro', False],
                      ['DYNC1H1', 'DYNC1H1_puro', False],
                      ['CTNNB1', 'CTNNB1_puro', False],
                      ['AURKA_puro','AURKA', False],
                      ['CTNNB1_LG007', 'CTNNB1_DMSO', False]]


for pairs in keys_distributions:
    print("############################################")
    welch_t_test(bigger_key=pairs[0], smaller_key=pairs[1], 
                 feature="nb_foci", data=df_30, 
                 two_tail=pairs[2], alpha=0.02)
    print("\n")

## Proportion mRNAs in foci

In [ ]:
keys_distributions = [['ASPM', 'ASPM_puro', False], 
                      ['BUB1', 'BUB1_puro', False],
                      ['DYNC1H1', 'DYNC1H1_puro', False],
                      ['CTNNB1', 'CTNNB1_puro', False],
                      ['AURKA_puro','AURKA', False],
                      ['CTNNB1_LG007', 'CTNNB1_DMSO', False]]


for pairs in keys_distributions:
    print("############################################")
    welch_t_test(bigger_key=pairs[0], smaller_key=pairs[1], 
                 feature="proportion_rna_in_foci", data=df_30, 
                 two_tail=pairs[2], alpha=0.02)
    print("\n")

# Supervised classification

## Fonctions

In [ ]:
def train_rf(data, pattern, features, max_train_pos=150, ratio_negative=4):
    np.random.seed(1)

    # get X and y annotated
    df_annotated = data.loc[data.loc[:, "annotated"], features + [pattern]]
    data_annotated = df_annotated.to_numpy(dtype=np.float32)
    X_annotated = data_annotated[:, :-1]
    y_annotated = data_annotated[:, -1]

    # balance annotated data (20% positive and 80% negative)
    nb_true = min(int(y_annotated.sum()), max_train_pos)
    mask_true = y_annotated.copy().astype(bool)
    nb_false = int(len(y_annotated) - np.count_nonzero(y_annotated))
    mask_false = ~y_annotated.copy().astype(bool)
    nb_false_to_keep = nb_true * ratio_negative

    # build a training dataset with the right proportions
    X_annotated_true = X_annotated[mask_true, :]
    X_annotated_false = X_annotated[mask_false, :]
    y_annotated_true = y_annotated[mask_true]
    y_annotated_false = y_annotated[mask_false]
    indices_annotated_true = np.array([i for i in range(len(X_annotated_true))], dtype=np.int64)
    indices_train_true = np.random.choice(indices_annotated_true, size=nb_true, replace=False)
    X_train_true = X_annotated_true[indices_train_true, :]
    y_train_true = y_annotated_true[indices_train_true]
    print("X_train_true:", X_train_true.shape)
    print("y_train_true:", y_train_true.shape)

    indices_annotated_false = np.array([i for i in range(len(X_annotated_false))], dtype=np.int64)
    indices_train_false = np.random.choice(indices_annotated_false, size=nb_false_to_keep, replace=False)
    X_train_false = X_annotated_false[indices_train_false, :]
    y_train_false = y_annotated_false[indices_train_false]
    print("X_train_false:", X_train_false.shape)
    print("y_train_false:", y_train_false.shape)

    X_train = np.concatenate((X_train_false, X_train_true))
    y_train = np.concatenate((y_train_false, y_train_true))
    indices_train = np.array([i for i in range(len(X_train))], dtype=np.int64)
    np.random.shuffle(indices_train)
    X_train = X_train[indices_train, :]
    y_train = y_train[indices_train].astype(np.int64)
    print("X_train:", X_train.shape)
    print("y_train:", y_train.shape, "\n")
    
    # train model
    scaler = StandardScaler()
    X_train_normalized = scaler.fit_transform(X_train)

    rf = RandomForestClassifier(n_estimators=100, criterion="entropy", 
                                max_depth=3, min_samples_split=2, max_features=10,
                                oob_score=True, n_jobs=8, random_state=13, verbose=0)
    rf.fit(X_train_normalized, y_train)
    
    # evaluate model
    oob_score = rf.oob_score_
    print("oob_score:", oob_score)
    feature_importance = rf.feature_importances_
    indices = np.argsort(feature_importance)[::-1][:5]
    print("feature importance:")
    for i in range(len(indices)):
        indice = indices[i]
        print("\t {0:.3f} {1}".format(feature_importance[indice], features[indice]))
        
    print()
    
    # dummy classifier
    model = DummyClassifier(strategy="most_frequent", random_state=None)
    model.fit(X_train_normalized, y_train)
    scores = cross_val_score(model, X_train_normalized, y_train, cv=5)
    print("dummy model mean CV score:", scores.mean())
    
    print()

    # accuracy 0-label
    indices_test_false = []
    for i in indices_annotated_false:
        if i not in indices_train_false:
            indices_test_false.append(i)
    indices_test_false = np.array(indices_test_false, dtype=np.int64)
    X_test_false = X_annotated_false[indices_test_false, :]
    X_test_false_normalized = scaler.transform(X_test_false)
    print("X_test_false:", X_test_false_normalized.shape)

    y_pred_false = rf.predict(X_test_false_normalized)
    y_true = np.array([0 for _ in range(len(X_test_false_normalized))], dtype=np.int64)
    print("nb false positive:", y_pred_false.sum())
    print("accuracy 0-label:", 1 - y_pred_false.sum() / len(y_pred_false))
    
    print()
    
    return scaler, rf

In [ ]:
def bootstrap_rf(data, pattern, features, max_train_pos=150, ratio_negative=4):
    np.random.seed(13)

    # get X and y annotated
    df_annotated = data.loc[data.loc[:, "annotated"], features + [pattern]]
    data_annotated = df_annotated.to_numpy(dtype=np.float32)
    X_annotated = data_annotated[:, :-1]
    y_annotated = data_annotated[:, -1]

    # balance annotated data (25% positive and 75% negative)
    nb_true = min(int(y_annotated.sum()), max_train_pos)
    mask_true = y_annotated.copy().astype(bool)
    nb_false = int(len(y_annotated) - np.count_nonzero(y_annotated))
    mask_false = ~y_annotated.copy().astype(bool)
    nb_false_to_keep = nb_true * ratio_negative

    # build a training dataset with the right proportions
    X_annotated_true = X_annotated[mask_true, :]
    X_annotated_false = X_annotated[mask_false, :]
    y_annotated_true = y_annotated[mask_true]
    y_annotated_false = y_annotated[mask_false]
    indices_annotated_true = np.array([i for i in range(len(X_annotated_true))], dtype=np.int64)
    indices_train_true = np.random.choice(indices_annotated_true, size=nb_true, replace=False)
    X_train_true = X_annotated_true[indices_train_true, :]
    y_train_true = y_annotated_true[indices_train_true]
    print("X_train_true:", X_train_true.shape)
    print("y_train_true:", y_train_true.shape)

    indices_annotated_false = np.array([i for i in range(len(X_annotated_false))], dtype=np.int64)
    indices_train_false = np.random.choice(indices_annotated_false, size=nb_false_to_keep, replace=False)
    X_train_false = X_annotated_false[indices_train_false, :]
    y_train_false = y_annotated_false[indices_train_false]
    print("X_train_false:", X_train_false.shape)
    print("y_train_false:", y_train_false.shape)

    X_train = np.concatenate((X_train_false, X_train_true))
    y_train = np.concatenate((y_train_false, y_train_true))
    indices_train = np.array([i for i in range(len(X_train))], dtype=np.int64)
    np.random.shuffle(indices_train)
    X_train = X_train[indices_train, :]
    y_train = y_train[indices_train].astype(np.int64)
    print("X_train:", X_train.shape)
    print("y_train:", y_train.shape, "\n")
    
    oob_scores = []
    for i in range(100):
        
        indices_train = np.array([i for i in range(len(X_train))], dtype=np.int64)
        nb_train_sample = len(indices_train)
        nb_train_sample_bootstrap = int(0.8 * nb_train_sample)
        indices_train_bootstrap = np.random.choice(indices_train, size=nb_train_sample_bootstrap, replace=False)
        X_train_bootstrap = X_train[indices_train_bootstrap, :]
        y_train_bootstrap = y_train[indices_train_bootstrap].astype(np.int64)
        
        # train model
        scaler = StandardScaler()
        X_train_bootstrap_normalized = scaler.fit_transform(X_train_bootstrap)

        rf = RandomForestClassifier(n_estimators=100, criterion="entropy", 
                                    max_depth=3, min_samples_split=2, max_features=10,
                                    oob_score=True, n_jobs=8, random_state=13, verbose=0)
        rf.fit(X_train_bootstrap_normalized, y_train_bootstrap)
    
        # evaluate model
        oob_score = rf.oob_score_
        oob_scores.append(oob_score)
    
    oob_scores = np.array(oob_scores, dtype=np.float32)
    print("mean oob_score:", oob_scores.mean(), "\n")
    
    return oob_scores

## Foci

In [ ]:
features = ["nb_foci", "proportion_rna_in_foci",
            "index_foci_mean_distance_cyt", "index_foci_mean_distance_nuc",
            "proportion_rna_in_nuc",
            "index_mean_distance_cyt", "index_mean_distance_nuc", 
            "index_rna_opening_30", "index_peripheral_dispersion",
            "index_rna_nuc_edge", "index_rna_nuc_radius_5_10", "index_rna_nuc_radius_10_15",
            "index_rna_cyt_radius_0_5", "index_rna_cyt_radius_5_10", "index_rna_cyt_radius_10_15"]

scaler, rf = train_rf(data=df_30_no_drug, 
                      pattern="pattern_foci", 
                      features=features)

df_all = df_30_no_drug.loc[:, features]
print("df_all:", df_all.shape)

X = df_all.to_numpy(dtype=np.float32)
X_normalized = scaler.transform(X)
print("X_normalized:", X_normalized.shape)

predictions = rf.predict(X_normalized)
print("predictions:", predictions.shape, predictions.dtype)
probabilities = rf.predict_proba(X_normalized)[:, 1]
print("probabilities:", probabilities.shape, probabilities.dtype)

df_30_no_drug.loc[:, "prediction_foci"] = predictions
df_30_no_drug.loc[:, "probability_foci"] = probabilities

In [ ]:
df_all = df_30.loc[:, features]
print("df_all:", df_all.shape)

X = df_all.to_numpy(dtype=np.float32)
X_normalized = scaler.transform(X)
print("X_normalized:", X_normalized.shape)

predictions = rf.predict(X_normalized)
print("predictions:", predictions.shape, predictions.dtype)
probabilities = rf.predict_proba(X_normalized)[:, 1]
print("probabilities:", probabilities.shape, probabilities.dtype)

df_30.loc[:, "prediction_foci"] = predictions
df_30.loc[:, "probability_foci"] = probabilities

In [ ]:
plot_barplot_foci("prediction_foci", df_30_no_drug,
                  path_output=os.path.join(plot_directory, "barplot_prediction_foci"), 
                  extension=["png", "pdf"])
plot_barplot_foci("probability_foci", df_30_no_drug,
                  path_output=os.path.join(plot_directory, "barplot_probability_foci"), 
                  extension=["png", "pdf"])
plot_barplot_foci_no_puro("prediction_foci", df_30_no_drug, 
                          path_output=os.path.join(plot_directory, "barplot_prediction_foci_no_puro"), 
                          extension=["png", "pdf"])

### Specific plots for the paper

In [ ]:
path_output = os.path.join(plot_directory, "plot_8b_boxplot_nb_foci")
extension = ["png", "pdf"]
feature = "nb_foci"

# parameters
foci = ["DYNC1H1", "DYNC1H1_puro", 
        "BUB1", "BUB1_puro", 
        "CTNNB1", "CTNNB1_puro",
        "ASPM", "ASPM_puro",
        "AURKA", "AURKA_puro"]
foci_no_puro = ["DYNC1H1", "BUB1", "CTNNB1", "ASPM", "AURKA"]
foci_puro = ["DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", "ASPM_puro", "AURKA_puro"] 

# plot
fig, ax = plt.subplots(figsize=(8, 3))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black', alpha=0.95)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="gene", y=feature, data=df_30_no_drug, hue="puromycin",
            palette=["#4381de", "#e83333"], order=["DYNC1H1", "BUB1", "CTNNB1", "ASPM", "AURKA"],
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0, ax=ax,)

# scatter plot no puro
data_ = df_30_no_drug.query("key in {0}".format(foci_no_puro))
int_gene = {}
for i, gene in enumerate(foci_no_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=-0.06, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.1)

# scatter plot puro
data_ = df_30_no_drug.query("key in {0}".format(foci_puro))
int_gene = {}
for i, gene in enumerate(foci_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=0.06, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.1)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 15)
patch_nopuro = mpatches.Patch(color="#4381de", label='No puromycin', alpha=0.95)
patch_puro = mpatches.Patch(color="#e83333", label='Puromycin', alpha=0.95)
plt.legend(handles=[patch_nopuro, patch_puro], loc='upper left', fontsize=15)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_8b_barplot_prediction_foci")
extension = ["png", "pdf"]
feature = "prediction_foci"

# parameters
foci = ["DYNC1H1", "DYNC1H1_puro", 
        "BUB1", "BUB1_puro", 
        "CTNNB1", "CTNNB1_puro",
        "ASPM", "ASPM_puro",
        "AURKA", "AURKA_puro"]
foci_no_puro = ["DYNC1H1", "BUB1", "CTNNB1", "ASPM", "AURKA"]
foci_puro = ["DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", "ASPM_puro", "AURKA_puro"]

# get data
mask_dync1h1 = (df_30_no_drug.loc[:, "gene"] == "DYNC1H1").astype(bool)
mask_bub1 = (df_30_no_drug.loc[:, "gene"] == "BUB1").astype(bool)
mask_ctnnb1 = (df_30_no_drug.loc[:, "gene"] == "CTNNB1").astype(bool)
mask_aspm = (df_30_no_drug.loc[:, "gene"] == "ASPM").astype(bool)
mask_aurka = (df_30_no_drug.loc[:, "gene"] == "AURKA").astype(bool)
mask = mask_dync1h1 | mask_bub1 | mask_ctnnb1 | mask_aspm | mask_aurka
data = df_30_no_drug.loc[mask, :]

# plot
fig, ax = plt.subplots(figsize=(8, 3))

# barplot
sns.barplot(x="gene", y=feature, hue="puromycin", data=data, 
            palette=["#4381de", "#e83333"], order=["DYNC1H1", "BUB1", "CTNNB1", "ASPM", "AURKA"], 
            estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
            errcolor='.26', errwidth=2, capsize=0.1, dodge=True, alpha=0.95,
            ax=ax)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 1)
patch_nopuro = mpatches.Patch(color="#4381de", label='No puromycin', alpha=0.95)
patch_puro = mpatches.Patch(color="#e83333", label='Puromycin', alpha=0.95)
plt.legend(handles=[patch_nopuro, patch_puro], loc='upper left', fontsize=15)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
    
# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_8b_boxplot_proportion_rna_foci")
extension = ["png", "pdf"]
feature = "proportion_rna_in_foci"

# parameters
foci = ["DYNC1H1", "DYNC1H1_puro", 
        "BUB1", "BUB1_puro", 
        "CTNNB1", "CTNNB1_puro",
        "ASPM", "ASPM_puro",
        "AURKA", "AURKA_puro"]
foci_no_puro = ["DYNC1H1", "BUB1", "CTNNB1", "ASPM", "AURKA"]
foci_puro = ["DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", "ASPM_puro", "AURKA_puro"] 

# plot
fig, ax = plt.subplots(figsize=(8, 3))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black', alpha=0.95)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="gene", y=feature, data=df_30_no_drug, hue="puromycin",
            palette=["#4381de", "#e83333"], order=["DYNC1H1", "BUB1", "CTNNB1", "ASPM", "AURKA"],
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0, ax=ax,)

# scatter plot no puro
data_ = df_30_no_drug.query("key in {0}".format(foci_no_puro))
int_gene = {}
for i, gene in enumerate(foci_no_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=-0.06, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.1)

# scatter plot puro
data_ = df_30_no_drug.query("key in {0}".format(foci_puro))
int_gene = {}
for i, gene in enumerate(foci_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=0.06, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.1)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 0.4)
patch_nopuro = mpatches.Patch(color="#4381de", label='No puromycin', alpha=0.95)
patch_puro = mpatches.Patch(color="#e83333", label='Puromycin', alpha=0.95)
plt.legend(handles=[patch_nopuro, patch_puro], loc='upper left', fontsize=15)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_s9g_boxplot_nb_foci")
extension = ["png", "pdf"]
feature = "nb_foci"

# parameters
genes = ["CTNNB1_DMSO", "CTNNB1_LG007"]

# plot
fig, ax = plt.subplots(figsize=(5, 3))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black')
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="key", y=feature, data=df_30,
            color="#4daf4a", order=genes,
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0, ax=ax,)

# scatter plot
data_ = df_30.query("key in {0}".format(genes))
int_gene = {}
for i, gene in enumerate(genes):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.4)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 6)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_s9g_boxplot_proportion_rna_foci")
extension = ["png", "pdf"]
feature = "proportion_rna_in_foci"

# parameters
genes = ["CTNNB1_DMSO", "CTNNB1_LG007"]

# get data
data = df_30.loc[~df_30.loc[:, "drug"].isna(), :]

# plot
fig, ax = plt.subplots(figsize=(5, 3))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black')
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="key", y=feature, data=df_30,
            color="#4daf4a", order=genes,
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0, ax=ax,)

# scatter plot
data_ = data.query("key in {0}".format(genes))
int_gene = {}
for i, gene in enumerate(genes):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.4)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 0.2)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_s9g_barplot_prediction_foci")
extension = ["png", "pdf"]
feature = "prediction_foci"

# parameters
genes = ["CTNNB1_DMSO", "CTNNB1_LG007"]

# get data
data = df_30.loc[~df_30.loc[:, "drug"].isna(), :]

# plot
fig, ax = plt.subplots(figsize=(5, 3))

# barplot
sns.barplot(x="key", y=feature, data=data, 
            color="#4daf4a", order=genes,
            estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
            errcolor='.26', errwidth=2, capsize=0.1, dodge=True, alpha=0.95,
            ax=ax)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 1)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
df_foci = df_30_no_drug.loc[df_30_no_drug.loc[:, "prediction_foci"].astype(bool), :]
print(df_foci.shape)
plot_boxplot_foci_no_puro(feature="proportion_rna_in_foci", data=df_foci, 
                          ylim=(0, 0.5), show_whisker=False, show_flier=False, 
                          random_flier=True, horizontal_line=None,
                          path_output=os.path.join(plot_directory, "boxplot_proportion_rna_in_foci_no_puro_class_foci"), 
                          extension=["png", "pdf"])

## Protrusion

In [ ]:
features = ["nb_foci", "proportion_rna_in_foci",
            "index_foci_mean_distance_cyt", "index_foci_mean_distance_nuc",
            "proportion_rna_in_nuc",
            "index_mean_distance_cyt", "index_mean_distance_nuc", 
            "index_rna_opening_30", "index_peripheral_dispersion",
            "index_rna_nuc_edge", "index_rna_nuc_radius_5_10", "index_rna_nuc_radius_10_15",
            "index_rna_cyt_radius_0_5", "index_rna_cyt_radius_5_10", "index_rna_cyt_radius_10_15"]

scaler, rf = train_rf(data=df_30_no_drug, 
                      pattern="pattern_protrusion", 
                      features=features)

df_all = df_30_no_drug.loc[:, features]
print("df_all:", df_all.shape)

X = df_all.to_numpy(dtype=np.float32)
X_normalized = scaler.transform(X)
print("X_normalized:", X_normalized.shape)

predictions = rf.predict(X_normalized)
print("predictions:", predictions.shape, predictions.dtype)
probabilities = rf.predict_proba(X_normalized)[:, 1]
print("probabilities:", probabilities.shape, probabilities.dtype)

df_30_no_drug.loc[:, "prediction_protrusion"] = predictions
df_30_no_drug.loc[:, "probability_protrusion"] = probabilities

In [ ]:
df_kif1c_ = df_kif1c.loc[:, features].copy()
X_kif1c = df_kif1c_.to_numpy(dtype=np.float32)
X_kif1c_normalized = scaler.transform(X_kif1c)
print("X_normalized:", X_kif1c_normalized.shape)
predictions_kif1c = rf.predict(X_kif1c_normalized)
print("predictions:", predictions_kif1c.shape, predictions_kif1c.dtype)
df_kif1c.loc[:, "prediction_protrusion"] = predictions_kif1c

In [ ]:
plot_barplot_protrusion("prediction_protrusion", df_30_no_drug, 
                        path_output=os.path.join(plot_directory, "barplot_prediction_protrusion"), 
                        extension=["png", "pdf"])
plot_barplot_protrusion("probability_protrusion", df_30_no_drug, 
                        path_output=os.path.join(plot_directory, "barplot_probability_protrusion"), 
                        extension=["png", "pdf"])

### Specific plots for the paper

In [ ]:
path_output = os.path.join(plot_directory, "plot_7b_boxplot_index_rna_opening_30")
extension = ["png", "pdf"]
feature = "index_rna_opening_30"

# parameters
protrusion = ["KIF1C", "KIF1C_puro", 
              "KIF4A", "KIF4A_puro", 
              "MYH3", "MYH3_puro"]
protrusion_no_puro = ["KIF1C", "KIF4A", "MYH3"]
protrusion_puro = ["KIF1C_puro", "KIF4A_puro", "MYH3_puro"]

# plot
fig, ax = plt.subplots(figsize=(8, 3))

# boxplot
boxprops = dict(linestyle='-', linewidth=2, edgecolor='black', alpha=0.95)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')
sns.boxplot(x="gene", y=feature, data=df_30_no_drug, hue="puromycin",
            palette=["#4381de", "#e83333"], order=["KIF1C", "KIF4A", "MYH3"],
            showmeans=True, meanline=False, meanprops=meanprops,
            boxprops=boxprops,
            showfliers=False, flierprops=flierprops, 
            medianprops=medianprops, 
            showcaps=False, capprops=capprops,
            whiskerprops=whiskerprops, whis=0, ax=ax,)

# scatter plot no puro
data_ = df_30_no_drug.query("key in {0}".format(protrusion_no_puro))
int_gene = {}
for i, gene in enumerate(protrusion_no_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=-0.06, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.15)

# scatter plot puro
data_ = df_30_no_drug.query("key in {0}".format(protrusion_puro))
int_gene = {}
for i, gene in enumerate(protrusion_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=0.06, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.15)

# text and lines
plt.axhline(y=1., c="red", lw=1, ls="dashed")

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 5)
patch_nopuro = mpatches.Patch(color="#4381de", label='No puromycin', alpha=0.95)
patch_puro = mpatches.Patch(color="#e83333", label='Puromycin', alpha=0.95)
plt.legend(handles=[patch_nopuro, patch_puro], loc='upper center', fontsize=15)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_7b_barplot_prediction_protrusion")
extension = ["png", "pdf"]
feature = "prediction_protrusion"

# parameters
protrusion = ["KIF1C", "KIF1C_puro", 
              "KIF4A", "KIF4A_puro", 
              "MYH3", "MYH3_puro"]
protrusion_no_puro = ["KIF1C", "KIF4A", "MYH3"]
protrusion_puro = ["KIF1C_puro", "KIF4A_puro", "MYH3_puro"]

# get data
mask_kif1c = (df_30_no_drug.loc[:, "gene"] == "KIF1C").astype(bool)
mask_kif4a = (df_30_no_drug.loc[:, "gene"] == "KIF4A").astype(bool)
mask_myh3 = (df_30_no_drug.loc[:, "gene"] == "MYH3").astype(bool)
mask = mask_kif1c | mask_kif4a | mask_myh3
data = df_30_no_drug.loc[mask, :]

# plot
fig, ax = plt.subplots(figsize=(8, 3))

# barplot
sns.barplot(x="gene", y=feature, hue="puromycin", data=data, 
            palette=["#4381de", "#e83333"], order=["KIF1C", "KIF4A", "MYH3"], 
            estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
            errcolor='.26', errwidth=2, capsize=0.1, dodge=True, alpha=0.95,
            ax=ax)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 1)
patch_nopuro = mpatches.Patch(color="#4381de", label='No puromycin', alpha=0.95)
patch_puro = mpatches.Patch(color="#e83333", label='Puromycin', alpha=0.95)
plt.legend(handles=[patch_nopuro, patch_puro], loc='upper center', fontsize=15)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
    
# show frame
plt.show()

In [ ]:
path_output = os.path.join(plot_directory, "plot_7b_barplot_proportion_rna_opening_30")
extension = ["png", "pdf"]
feature = "proportion_rna_opening_30"

# parameters
protrusion = ["KIF1C", "KIF1C_puro", 
              "KIF4A", "KIF4A_puro", 
              "MYH3", "MYH3_puro"]
protrusion_no_puro = ["KIF1C", "KIF4A", "MYH3"]
protrusion_puro = ["KIF1C_puro", "KIF4A_puro", "MYH3_puro"]

# get data
mask_kif1c = (df_30_no_drug.loc[:, "gene"] == "KIF1C").astype(bool)
mask_kif4a = (df_30_no_drug.loc[:, "gene"] == "KIF4A").astype(bool)
mask_myh3 = (df_30_no_drug.loc[:, "gene"] == "MYH3").astype(bool)
mask = mask_kif1c | mask_kif4a | mask_myh3
data = df_30_no_drug.loc[mask, :]

# plot
fig, ax = plt.subplots(figsize=(8, 3))

# barplot
sns.barplot(x="gene", y=feature, hue="puromycin", data=data, 
            palette=["#4381de", "#e83333"], order=["KIF1C", "KIF4A", "MYH3"], 
            estimator=np.mean, ci=95, n_boot=1000, units=None, orient="v",  
            errcolor='.26', errwidth=2, capsize=0.1, dodge=True, alpha=0.95,
            ax=ax)

# scatter plot no puro
data_ = df_30_no_drug.query("key in {0}".format(protrusion_no_puro))
int_gene = {}
for i, gene in enumerate(protrusion_no_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=-0.34, high=-0.06, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.2)

# scatter plot puro
data_ = df_30_no_drug.query("key in {0}".format(protrusion_puro))
int_gene = {}
for i, gene in enumerate(protrusion_puro):
    int_gene[gene] = i
values_x = data_.apply(lambda row: int_gene[row["key"]], axis=1)
values_x = np.random.uniform(low=0.06, high=0.34, size=len(values_x)) + values_x
values_y = data_.loc[:, feature]
plt.scatter(x=values_x, y=values_y, c='black', s=10, alpha=0.2)

# axes
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 0.15)
patch_nopuro = mpatches.Patch(color="#4381de", label='No puromycin', alpha=0.95)
patch_puro = mpatches.Patch(color="#e83333", label='Puromycin', alpha=0.95)
plt.legend(handles=[patch_nopuro, patch_puro], loc='upper center', fontsize=15)
plt.tight_layout()

# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
    
# show frame
plt.show()

## Nuclear

In [ ]:
features = ["nb_foci", "proportion_rna_in_foci",
            "index_foci_mean_distance_cyt", "index_foci_mean_distance_nuc",
            "proportion_rna_in_nuc",
            "index_mean_distance_cyt", "index_mean_distance_nuc", 
            "index_rna_opening_30", "index_peripheral_dispersion",
            "index_rna_nuc_edge", "index_rna_nuc_radius_5_10", "index_rna_nuc_radius_10_15",
            "index_rna_cyt_radius_0_5", "index_rna_cyt_radius_5_10", "index_rna_cyt_radius_10_15"]

scaler, rf = train_rf(data=df_30_no_drug, 
                      pattern="pattern_nuclear", 
                      features=features)

df_all = df_30_no_drug.loc[:, features]
print("df_all:", df_all.shape)

X = df_all.to_numpy(dtype=np.float32)
X_normalized = scaler.transform(X)
print("X_normalized:", X_normalized.shape)

predictions = rf.predict(X_normalized)
print("predictions:", predictions.shape, predictions.dtype)
probabilities = rf.predict_proba(X_normalized)[:, 1]
print("probabilities:", probabilities.shape, probabilities.dtype)

df_30_no_drug.loc[:, "prediction_nuclear"] = predictions
df_30_no_drug.loc[:, "probability_nuclear"] = probabilities

In [ ]:
plot_barplot_nuclear("prediction_nuclear", df_30_no_drug, 
                     path_output=os.path.join(plot_directory, "barplot_prediction_nuclear"), 
                     extension=["png", "pdf"])
plot_barplot_nuclear("probability_nuclear", df_30_no_drug, 
                     path_output=os.path.join(plot_directory, "barplot_probability_nuclear"), 
                     extension=["png", "pdf"])

## Perinuclear

In [ ]:
features = ["nb_foci", "proportion_rna_in_foci",
            "index_foci_mean_distance_cyt", "index_foci_mean_distance_nuc",
            "proportion_rna_in_nuc",
            "index_mean_distance_cyt", "index_mean_distance_nuc", 
            "index_rna_opening_30", "index_peripheral_dispersion",
            "index_rna_nuc_edge", "index_rna_nuc_radius_5_10", "index_rna_nuc_radius_10_15",
            "index_rna_cyt_radius_0_5", "index_rna_cyt_radius_5_10", "index_rna_cyt_radius_10_15"]

scaler, rf = train_rf(data=df_30_no_drug, 
                      pattern="pattern_perinuclear", 
                      features=features)

df_all = df_30_no_drug.loc[:, features]
print("df_all:", df_all.shape)

X = df_all.to_numpy(dtype=np.float32)
X_normalized = scaler.transform(X)
print("X_normalized:", X_normalized.shape)

predictions = rf.predict(X_normalized)
print("predictions:", predictions.shape, predictions.dtype)
probabilities = rf.predict_proba(X_normalized)[:, 1]
print("probabilities:", probabilities.shape, probabilities.dtype)

df_30_no_drug.loc[:, "prediction_perinuclear"] = predictions
df_30_no_drug.loc[:, "probability_perinuclear"] = probabilities

In [ ]:
plot_barplot_nuclear("prediction_perinuclear", df_30_no_drug,
                     path_output=os.path.join(plot_directory, "barplot_prediction_perinuclear"), 
                     extension=["png", "pdf"])
plot_barplot_nuclear("probability_perinuclear", df_30_no_drug, 
                     path_output=os.path.join(plot_directory, "barplot_probability_perinuclear"), 
                     extension=["png", "pdf"])

## Intranuclear

In [ ]:
features = ["nb_foci", "proportion_rna_in_foci",
            "index_foci_mean_distance_cyt", "index_foci_mean_distance_nuc",
            "proportion_rna_in_nuc",
            "index_mean_distance_cyt", "index_mean_distance_nuc", 
            "index_rna_opening_30", "index_peripheral_dispersion",
            "index_rna_nuc_edge", "index_rna_nuc_radius_5_10", "index_rna_nuc_radius_10_15",
            "index_rna_cyt_radius_0_5", "index_rna_cyt_radius_5_10", "index_rna_cyt_radius_10_15"]

scaler, rf = train_rf(data=df_30_no_drug, 
                      pattern="pattern_intranuclear", 
                      features=features)

df_all = df_30_no_drug.loc[:, features]
print("df_all:", df_all.shape)

X = df_all.to_numpy(dtype=np.float32)
X_normalized = scaler.transform(X)
print("X_normalized:", X_normalized.shape)

predictions = rf.predict(X_normalized)
print("predictions:", predictions.shape, predictions.dtype)
probabilities = rf.predict_proba(X_normalized)[:, 1]
print("probabilities:", probabilities.shape, probabilities.dtype)

df_30_no_drug.loc[:, "prediction_intranuclear"] = predictions
df_30_no_drug.loc[:, "probability_intranuclear"] = probabilities

In [ ]:
plot_barplot_nuclear("prediction_intranuclear", df_30_no_drug, 
                     path_output=os.path.join(plot_directory, "barplot_prediction_intranuclear"), 
                     extension=["png", "pdf"])
plot_barplot_nuclear("probability_intranuclear", df_30_no_drug, 
                     path_output=os.path.join(plot_directory, "barplot_probability_intranuclear"), 
                     extension=["png", "pdf"])

## Random

In [ ]:
mask_pattern_detected = (df_30_no_drug.loc[:, "prediction_foci"] | df_30_no_drug.loc[:, "prediction_intranuclear"]
                         | df_30_no_drug.loc[:, "prediction_nuclear"] | df_30_no_drug.loc[:, "prediction_perinuclear"]
                         | df_30_no_drug.loc[:, "prediction_protrusion"])
df_30_no_drug.loc[:, "prediction_random"] = ~mask_pattern_detected.astype(bool)

In [ ]:
def probability_random(row):
    probability_foci = row["probability_foci"]
    probability_nuclear = row["probability_nuclear"]
    probability_perinuclear = row["probability_perinuclear"]
    probability_protrusion = row["probability_protrusion"]
    probability_intranuclear = row["probability_intranuclear"]
    max_proba = max(probability_foci, probability_nuclear, probability_perinuclear, 
                    probability_protrusion, probability_intranuclear)
    probability_random = 1 - max_proba
    
    return probability_random

df_30_no_drug.loc[:, "probability_random"] = df_30_no_drug.apply(probability_random, axis=1)

In [ ]:
plot_barplot_general("prediction_random", df_30_no_drug, 
                     path_output=os.path.join(plot_directory, "barplot_prediction_random"), 
                     extension=["png", "pdf"])
plot_barplot_general("probability_random", df_30_no_drug, 
                     path_output=os.path.join(plot_directory, "barplot_probability_random"), 
                     extension=["png", "pdf"])

## Classification robustness

In [ ]:
features = ["nb_foci", "proportion_rna_in_foci",
            "index_foci_mean_distance_cyt", "index_foci_mean_distance_nuc",
            "proportion_rna_in_nuc",
            "index_mean_distance_cyt", "index_mean_distance_nuc", 
            "index_rna_opening_30", "score_polarization_cyt", "score_polarization_nuc",
            "index_peripheral_dispersion", "index_dispersion",
            "index_rna_nuc_edge", "index_rna_nuc_radius_5_10", "index_rna_nuc_radius_10_15",
            "index_rna_cyt_radius_0_5", "index_rna_cyt_radius_5_10", "index_rna_cyt_radius_10_15"]

patterns = ["pattern_foci", "pattern_protrusion", "pattern_intranuclear", "pattern_nuclear", "pattern_perinuclear"]

oob_scores = []
for pattern in patterns:
    print("###", pattern, "###")
    oob_scores_ = bootstrap_rf(data=df_30_no_drug, 
                               pattern=pattern, 
                               features=features)
    oob_scores.append(oob_scores_)

In [ ]:
path_output=os.path.join(plot_directory, "boxplot_rf")
extension=["png", "pdf"]

plt.figure(figsize=(10, 3))

# parameters
boxprops = dict(linestyle='-', linewidth=2)
flierprops = dict(marker='.', markerfacecolor='gray', markersize=5, markeredgecolor='gray')
medianprops = dict(linestyle='-', linewidth=2, color='black')
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
capprops = dict(linestyle='-', linewidth=1.5, color='grey')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='grey')

# boxplot
boxes = plt.boxplot(x=oob_scores,
                    patch_artist=True,
                    meanline=False, showmeans=True, showfliers=False, 
                    showbox=True, showcaps=False, whis=0,
                    boxprops=boxprops, flierprops=flierprops, medianprops=medianprops, 
                    meanprops=meanprops, capprops=capprops, whiskerprops=whiskerprops) 
 
for patch in boxes['boxes']:
    patch.set_facecolor("#4daf4a")
        
# scatter plot
for i, oob_scores_ in enumerate(oob_scores):
    values_y = oob_scores_
    values_x = np.array([i + 1 for _ in range(len(values_y))], dtype=np.float32)
    values_x = np.random.uniform(low=-0.25, high=0.25, size=len(values_x)) + values_x
    plt.scatter(x=values_x, y=values_y, c='gray', s=10, alpha=0.1)

# axes
plt.axhline(y=4/5, c="red", lw=2, ls="-")
plt.axhline(y=1, c="black", lw=1, ls="dashed")
plt.axhline(y=0, c="black", lw=1, ls="dashed")
plt.ylim((0.75, 1.01))
x_labels = ["foci", "protrusion", "intranuclear", "nuclear edge", "perinuclear"]
x_ticks = [i for i in range(1, len(x_labels) + 1)]
plt.xticks(ticks=x_ticks, labels=x_labels, rotation=0, fontweight="bold", fontsize=14)
plt.yticks(fontweight="bold", fontsize=14)
plt.figtext(x=0.3, y=0.2, s="Dummy classifier (accuracy score: 0.80)", fontweight="bold", fontsize=14, color="red")
plt.xlabel("")
plt.ylabel("")
plt.tight_layout()
    
# save frame
for extension_ in extension:
    path_output_ = path_output + "." + extension_
    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")

# show frame
plt.show()

# T-SNE

In [ ]:
features = ["nb_foci", "proportion_rna_in_foci",
            "index_foci_mean_distance_cyt", "index_foci_mean_distance_nuc",
            "proportion_rna_in_nuc",
            "index_mean_distance_cyt", "index_mean_distance_nuc", 
            "index_rna_opening_30", "index_peripheral_dispersion",
            "index_rna_nuc_edge", "index_rna_nuc_radius_5_10", "index_rna_nuc_radius_10_15",
            "index_rna_cyt_radius_0_5", "index_rna_cyt_radius_5_10", "index_rna_cyt_radius_10_15"]
            
df_features = df_30_no_drug.loc[:, features]
print(df_features.shape)
df_features.head()

In [ ]:
features = df_features.to_numpy(dtype=np.float32)
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)
print(normalized_features.shape, normalized_features.dtype)

In [ ]:
tsne = TSNE(n_components=2, n_iter=1000, metric="euclidean", init="pca", verbose=1, perplexity=30, random_state=13)
embedding_2d_tsne_features = tsne.fit_transform(normalized_features)
print(embedding_2d_tsne_features.shape)

In [ ]:
# keys
keys = list(df_30_no_drug["key"])
unique_keys = list(set(keys))
encoder = LabelEncoder()
encoder.fit(unique_keys)
keys_num = encoder.transform(keys)
print(unique_keys)

# labels
labels = list(df_30_no_drug["label"])
unique_labels = list(set(labels))
encoder_label = LabelEncoder()
encoder_label.fit(unique_labels)
labels_num = encoder_label.transform(labels)
unique_labels = unique_labels[1:]
print(unique_labels)

In [ ]:
def plot_embedding_annotation(embedding_2d, labels_num, figsize=(12, 12), legend="inside", 
                              path_output=None, extension=None):
    
    # parameters
    colors = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#a65628"]
    patterns_name = ['intranuclear', 'nuclear', 'perinuclear', "protrusion", 'foci', "random"]
    default_color = "#d9d9d9"
    
    x = embedding_2d.copy()
    y = labels_num.copy()

    # plot
    plt.figure(figsize=figsize)
    plt.scatter(x[:, 0], x[:, 1], label="unlabelled",
                c=default_color, alpha=0.7,
                marker="o", s=50)
    
    for i_pattern, pattern_name in enumerate(patterns_name):
        colors_pattern = colors[i_pattern]
        label_num = encoder_label.transform([pattern_name])[0]
        plt.scatter(x[y == label_num, 0], x[y == label_num, 1], label=pattern_name,
                    c=colors_pattern, alpha=1, 
                    marker="o", s=150)

    # legend and ticks
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    if legend == "inside":
        plt.legend(prop={'size': 15}, loc='lower right')
    elif legend is None:
        pass
    else:
        plt.legend(prop={'size': 15}, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.axis("off")
    plt.tight_layout()
    
    # save frame
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
            
    # show frame
    plt.show()

    return

In [ ]:
plot_embedding_annotation(embedding_2d=embedding_2d_tsne_features, 
                          labels_num=labels_num, 
                          legend=None, 
                          figsize=(7, 7), 
                          path_output=os.path.join(plot_directory, "tsne_annotation_nolegend_small"),
                          extension=["png", "pdf"])
plot_embedding_annotation(embedding_2d=embedding_2d_tsne_features, 
                          labels_num=labels_num, 
                          legend=None, 
                          path_output=os.path.join(plot_directory, "tsne_annotation_nolegend"),
                          extension=["png", "pdf"])
plot_embedding_annotation(embedding_2d=embedding_2d_tsne_features, 
                          labels_num=labels_num, 
                          legend="outside",
                          figsize=(15, 10), 
                          path_output=os.path.join(plot_directory, "tsne_annotation_legend"),
                          extension=["png", "pdf"])

In [ ]:
def plot_embedding_prediction(embedding_2d, data, path_output=None, extension=None):
    
    # parameters
    colors = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#a65628"]
    patterns_name = ['intranuclear', 'nuclear', 'perinuclear', "protrusion", 'foci', "random"]
    default_color = "#d9d9d9"
    predictions = ['prediction_intranuclear', 'prediction_nuclear', 'prediction_perinuclear', 
                   'prediction_protrusion', 'prediction_foci', 'prediction_random']
    pattern_names = ["intranuclear detected", "nuclear detected", "perinuclear detected", 
                     "protrusion detected", "foci detected", "random detected"]
    x = embedding_2d.copy()
    
    for i_pattern, prediction in enumerate(predictions):
        colors_pattern = colors[i_pattern]
        mask = data.loc[:, prediction].astype(bool)
        pattern_name = pattern_names[i_pattern]        
        
        # plot
        plt.figure(figsize=(7, 7))
        plt.scatter(x[:, 0], x[:, 1],
                    c=default_color, alpha=0.7, 
                    marker="o", s=50)
        plt.scatter(x[mask, 0], x[mask, 1], label=pattern_name,
                    c=colors_pattern, alpha=1, 
                    marker="o", s=150)
    
        # legend and ticks
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        plt.axis('off')
        plt.tight_layout()

        # save frame
        if path_output is not None and extension is not None:
            if isinstance(extension, str):
                path_output_ = path_output + "_" + pattern_name.split(" ")[0] + "." + extension 
                plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
            elif isinstance(extension, list):
                for extension_ in extension:
                    path_output_ = path_output + "_" + pattern_name.split(" ")[0] + "." + extension_
                    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
        
        # show frame
        plt.show()

    return

In [ ]:
plot_embedding_prediction(embedding_2d=embedding_2d_tsne_features, 
                          data=df_30_no_drug, 
                          path_output=os.path.join(plot_directory, "tsne_prediction"),
                          extension=["png", "pdf"])

In [ ]:
def plot_embedding_prediction_probability(embedding_2d, data, colorbar=True,
                                          path_output=None, extension=None):
    
    # parameters
    probabilities = ['probability_intranuclear', 'probability_nuclear', 'probability_perinuclear', 
                     'probability_protrusion', 'probability_foci', "probability_random"]
    pattern_names = ["intranuclear detected", "nuclear detected", "perinuclear detected", 
                     "protrusion detected", "foci detected", "random detected"]
    colors = ["Reds", "Blues", "Greens", "Purples", "Oranges", "Greys"]
    x = embedding_2d.copy()
    
    for i_pattern, probability in enumerate(probabilities):
        pattern_name = pattern_names[i_pattern]
        color = colors[i_pattern]
        data_sorted = data.copy(deep=True)
        data_sorted = data_sorted.reset_index(inplace=False, drop=True)
        data_sorted = data_sorted.sort_values(by=probability, axis=0, ascending=True, 
                                              inplace=False, kind='quicksort', na_position='last')
        alpha_values = data_sorted.loc[:, probability]
        id_sorted = data_sorted.index
        x_sorted = x[id_sorted, :]

        # plot
        plt.figure(figsize=(7, 7))
        plt.scatter(x_sorted[:, 0], x_sorted[:, 1],
                    c=alpha_values, cmap=color,
                    marker="o", s=50)
        
        # legend and ticks
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        if colorbar:
            plt.colorbar()
            plt.clim((0, 1))
        plt.axis('off')
        plt.tight_layout()
        
        # save frame
        if path_output is not None and extension is not None:
            if isinstance(extension, str):
                path_output_ = path_output + "_" + pattern_name.split(" ")[0] + "." + extension 
                plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
            elif isinstance(extension, list):
                for extension_ in extension:
                    path_output_ = path_output + "_" + pattern_name.split(" ")[0] + "." + extension_
                    plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
            
        # show frame
        plt.show()

    return

In [ ]:
plot_embedding_prediction_probability(embedding_2d=embedding_2d_tsne_features, 
                                      data=df_30_no_drug,
                                      colorbar=False, 
                                      path_output=os.path.join(plot_directory, "tsne_probability_nocolorbar"),
                                      extension=["png", "pdf"])

In [ ]:
def plot_embedding_gene(embedding_2d, data, output_dir=None, extension=None):
    
    # parameters
    color = "#4381de"
    color_puro = "#e83333"
    default_color = "#d9d9d9"
    
    x = embedding_2d.copy()
    genes = set(data.loc[:, "gene"])
    for i_gene, gene in enumerate(genes):
        mask_gene = (data.loc[:, "gene"] == gene).astype(bool)
        data_gene = data.loc[mask_gene, :]
        x_gene = x[mask_gene, :]
        mask_puro = data_gene.loc[:, "puromycin"].astype(bool)
        print(mask_puro.shape, mask_puro.sum())
        
        # plot
        plt.figure(figsize=(7, 7))
        plt.scatter(x[:, 0], x[:, 1], label="other",
                    c=default_color, alpha=0.7, 
                    marker="o", s=50)
        if mask_puro.sum() > 0:
            plt.scatter(x_gene[mask_puro, 0], x_gene[mask_puro, 1], label=gene + " puromycin",
                        c=color_puro, alpha=1, edgecolor=None,
                        marker="o", s=50)
        plt.scatter(x_gene[~mask_puro, 0], x_gene[~mask_puro, 1], label=gene,
                    c=color, alpha=1, edgecolor=None,
                    marker="o", s=50)    
    
        # legend and ticks
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        plt.axis('off')
        plt.legend()
        plt.tight_layout()

        # save frame
        if output_dir is not None and extension is not None:
            if isinstance(extension, str):
                path_output = os.path.join(output_dir, "tsne_gene_{0}.{1}".format(gene, extension))
                plt.savefig(path_output, format=extension, bbox_inches="tight", dpi="figure")
            elif isinstance(extension, list):
                for extension_ in extension:
                    path_output = os.path.join(output_dir, "tsne_gene_{0}.{1}".format(gene, extension_))
                    plt.savefig(path_output, format=extension_, bbox_inches="tight", dpi="figure")
    
        # show frame
        plt.show()

    return

In [ ]:
plot_embedding_gene(embedding_2d=embedding_2d_tsne_features, 
                          data=df_30_no_drug, 
                          output_dir=plot_directory,
                          extension=["png", "pdf"])

# Heatmap

## Masks

In [ ]:
patterns = ["intranuclear", "perinuclear", "nuclear", "foci", "protrusion"]
genes_control_all = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a pattern")
print("# - H0 %control == %cell")
print("# - H1 %control != %cell")
print("# - H0 rejected if p-value >= 0.001")
print("################################################", "\n")

for pattern in patterns:
    print("####################################")
    print("Pattern:", pattern, "\n")
    prediction_feature = "prediction_" + pattern
    prediction_values = df_30_no_drug.loc[:, prediction_feature]

    # gene values
    for gene in genes_control_all:
        genes_control = [i for i in genes_control_all if i != gene]
        
        # mask control
        mask_control = df_30_no_drug.loc[:, "gene"].isin(genes_control)
        mask_not_annotated = ~df_30_no_drug.loc[:, "annotated"]
        mask_control &= mask_not_annotated
    
        # control values
        prediction_values_control = prediction_values[mask_control]
        nb_control = len(prediction_values_control)
        nb_control_pattern = prediction_values_control.sum()
        nb_control_nopattern = nb_control - nb_control_pattern
    
        # mask target and values
        mask_gene = (df_30_no_drug.loc[:, "gene"] == gene).astype(bool)
        prediction_values_gene = prediction_values[mask_gene]
        nb_target = len(prediction_values_gene)
        nb_target_pattern = prediction_values_gene.sum()
        nb_target_nopattern = nb_target - nb_target_pattern
        
        # contingency table
        table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])
        proportion_target = (nb_target_pattern / nb_target) * 100
        proportion_control = (nb_control_pattern / nb_control) * 100
        
        # perform a two-sided Fisher test
        alpha = 0.001
        oddsratio, pvalue = fisher_exact(table, alternative="two-sided")
        
        print("{0} vs {1}".format(gene, "Control"))
        print("  {0}: {1} cells | {2:.2f}% {3} recognized".format(gene, nb_target, proportion_target, pattern))
        print("  {0}: {1} cells | {2:.2f}% {3} recognized".format("Control", nb_control, proportion_control, pattern))

        if pvalue > alpha:
            print("  => {0} not significantly different than {1} (p-value:{2:.3f} | odd-ratio:{3:.3f})"
                  .format(gene, "Control", pvalue, oddsratio))
        else:
            print("  => {0} significantly different than {1} (p-value:{2:.3f} | odd-ratio:{3:.3f})"
                  .format(gene, "Control", pvalue, oddsratio))
        print()
    print()

In [ ]:
patterns = ["intranuclear", "perinuclear", "nuclear", "foci", "protrusion"]
genes_control_all = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a pattern")
print("# - H0 %control == %cell")
print("# - H1 %control != %cell")
print("# - H0 rejected if p-value >= 0.05")
print("################################################", "\n")

for pattern in patterns:
    print("####################################")
    print("Pattern:", pattern, "\n")
    prediction_feature = "prediction_" + pattern
    prediction_values = df_30_no_drug.loc[:, prediction_feature]

    # gene values
    for gene in genes_control_all:
        genes_control = [i for i in genes_control_all if i != gene]
        
        # mask control
        mask_control = df_30_no_drug.loc[:, "gene"].isin(genes_control)
        mask_not_annotated = ~df_30_no_drug.loc[:, "annotated"]
        mask_control &= mask_not_annotated
    
        # control values
        prediction_values_control = prediction_values[mask_control]
        nb_control = len(prediction_values_control)
        nb_control_pattern = prediction_values_control.sum()
        nb_control_nopattern = nb_control - nb_control_pattern
    
        # mask target and values
        mask_gene = (df_30_no_drug.loc[:, "gene"] == gene).astype(bool)
        prediction_values_gene = prediction_values[mask_gene]
        nb_target = len(prediction_values_gene)
        nb_target_pattern = prediction_values_gene.sum()
        nb_target_nopattern = nb_target - nb_target_pattern
        
        # contingency table
        table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])
        proportion_target = (nb_target_pattern / nb_target) * 100
        proportion_control = (nb_control_pattern / nb_control) * 100
        
        # perform a two-sided Fisher test
        alpha = 0.05
        oddsratio, pvalue = fisher_exact(table, alternative="two-sided")
        
        print("{0} vs {1}".format(gene, "Control"))
        print("  {0}: {1} cells | {2:.2f}% {3} recognized".format(gene, nb_target, proportion_target, pattern))
        print("  {0}: {1} cells | {2:.2f}% {3} recognized".format("Control", nb_control, proportion_control, pattern))

        if pvalue > alpha:
            print("  => {0} not significantly different than {1} (p-value:{2:.3f} | odd-ratio:{3:.3f})"
                  .format(gene, "Control", pvalue, oddsratio))
        else:
            print("  => {0} significantly different than {1} (p-value:{2:.3f} | odd-ratio:{3:.3f})"
                  .format(gene, "Control", pvalue, oddsratio))
        print()
    print()

In [ ]:
patterns = ["intranuclear", "perinuclear", "nuclear", "foci", "protrusion"]
genes = ["MYH3", "CEP192",
         "ATP6A2", "AP1S2", "AKAP9", "AKAP1", "HSP90B1",
         "SPEN", "ASPM",
         "DYNC1H1", "BUB1", "CTNNB1", "HMMR", "CEP170P1", "CRKL", "PAK2", "AURKA",
         "KIF1C", "KIF4A", "RAB13", "DYNLL2", "KIF5B",
         "FLNA"]
genes_control = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

# mask heatmap and annotations
mask = np.zeros((len(genes), len(patterns))).astype(bool)
annotations = [[0 for _ in patterns] for _ in genes]
annotations = np.array(annotations, dtype=np.float32)

# mask control
mask_control = df_30_no_drug.loc[:, "gene"].isin(genes_control)
mask_not_annotated = ~df_30_no_drug.loc[:, "annotated"]
mask_control &= mask_not_annotated

for col, pattern in enumerate(patterns):
    print("####################################")
    print("Pattern:", pattern, "\n")
    prediction_feature = "prediction_" + pattern
    prediction_values = df_30_no_drug.loc[:, prediction_feature]
    
    # control values
    prediction_values_control = prediction_values[mask_control]
    nb_control = len(prediction_values_control)
    nb_control_pattern = prediction_values_control.sum()
    nb_control_nopattern = nb_control - nb_control_pattern
    
    # gene values
    for row, gene in enumerate(genes):
        mask_gene = (df_30_no_drug.loc[:, "gene"] == gene).astype(bool)
        prediction_values_gene = prediction_values[mask_gene]
        nb_target = len(prediction_values_gene)
        nb_target_pattern = prediction_values_gene.sum()
        nb_target_nopattern = nb_target - nb_target_pattern
        table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])
        
        # perform a one-sided Fisher test
        alpha = 0.05
        oddsratio, pvalue = fisher_exact(table, alternative="greater")
        annotations[row, col] = pvalue
        
        if pvalue > alpha:
            mask[row, col] = True
            print("\t", gene, "Not significantly higher", "\t", alpha, oddsratio, pvalue)
        else:
            print("\t", gene, "Significantly higher", "\t", alpha, oddsratio, pvalue)
    print()

print(annotations.shape, annotations.dtype)
print(mask.shape, mask.dtype)

## Plot

In [ ]:
# collect predictions proportions
df_heatmap = df_30_no_drug.loc[df_30_no_drug.loc[:, "puromycin"] == 0, :]
df_heatmap = df_heatmap.loc[:, ["cell", "key", 
                                "prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                "prediction_foci", "prediction_protrusion"]]
nb_cells_per_key = df_heatmap.groupby("key")["cell"].count()
df_heatmap = df_heatmap.groupby(by="key")["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                          "prediction_foci", "prediction_protrusion"].sum()
df_heatmap = df_heatmap.merge(right=nb_cells_per_key, how="inner", on="key")
df_heatmap.loc[:, "prediction_foci"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_protrusion"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_nuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_perinuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_intranuclear"] /= df_heatmap.loc[:, "cell"]

# sort genes
genes = ["MYH3", "CEP192",
         "ATP6A2", "AP1S2", "AKAP9", "AKAP1", "HSP90B1",
         "SPEN", "ASPM",
         "DYNC1H1", "BUB1", "CTNNB1", "HMMR", "CEP170P1", "CRKL", "PAK2", "AURKA",
         "KIF1C", "KIF4A", "RAB13", "DYNLL2", "KIF5B",
         "FLNA",
         "KIF20B", "MYO18A", "MYSNE2", "PLEC"]
df_heatmap = df_heatmap.T.loc[:, genes]

# sort patterns
df_heatmap = df_heatmap.T
features_name = ["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                 "prediction_foci", "prediction_protrusion"]
df_heatmap = df_heatmap.loc[:, features_name]
print(df_heatmap.shape)

# build numpy matrix
patterns = ["intranuclear", "perinuclear", "nuclear edge", "foci", "protrusion"]
features_heatmap = df_heatmap.to_numpy(dtype=np.float32)
features_heatmap = np.round(features_heatmap, decimals=2)
print(features_heatmap.shape, features_heatmap.dtype)
df_heatmap

In [ ]:
def plot_heatmap_gene(data, genes, patterns, figsize=(15, 5), colorbar=True,
                      path_output=None, extension=None):
    # plot
    plt.figure(figsize=figsize)

    # heatmap
    ax = sns.heatmap(data=data,
                     vmin=0, vmax=1, 
                     cmap="Reds", cbar=colorbar, 
                     xticklabels=patterns, yticklabels=genes)
    ax.set_facecolor("#f7fbff")

    # axes
    plt.ylim((27, 0))
    plt.xticks([])
    plt.yticks(rotation=0, fontsize=15, fontweight="bold")
    plt.tight_layout()

    # save frame    
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
                
    # show frame
    plt.show()
    
    return

In [ ]:
plot_heatmap_gene(data=features_heatmap, 
                  genes=genes, 
                  patterns=patterns, 
                  figsize=(5, 10), 
                  colorbar=True,
                  path_output=os.path.join(plot_directory, "heatmap_genes_colorbar"),
                  extension=["png", "pdf"])

In [ ]:
# collect predictions proportions
df_heatmap = df_30_no_drug.loc[df_30_no_drug.loc[:, "puromycin"] == 0, :]
df_heatmap = df_heatmap.loc[:, ["cell", "key", 
                                "prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                "prediction_foci", "prediction_protrusion"]]
nb_cells_per_key = df_heatmap.groupby("key")["cell"].count()
df_heatmap = df_heatmap.groupby(by="key")["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                          "prediction_foci", "prediction_protrusion"].sum()
df_heatmap = df_heatmap.merge(right=nb_cells_per_key, how="inner", on="key")
df_heatmap.loc[:, "prediction_foci"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_protrusion"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_nuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_perinuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_intranuclear"] /= df_heatmap.loc[:, "cell"]

# sort genes
genes = ["MYH3", "CEP192",
         "ATP6A2", "AP1S2", "AKAP9", "AKAP1", "HSP90B1",
         "SPEN", "ASPM",
         "DYNC1H1", "BUB1", "CTNNB1", "HMMR", "CEP170P1", "CRKL", "PAK2", "AURKA",
         "KIF1C", "KIF4A", "RAB13", "DYNLL2", "KIF5B",
         "FLNA"]
df_heatmap = df_heatmap.T.loc[:, genes]

# sort patterns
df_heatmap = df_heatmap.T
features_name = ["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                 "prediction_foci", "prediction_protrusion"]
df_heatmap = df_heatmap.loc[:, features_name]
print(df_heatmap.shape)

# build numpy matrix
patterns = ["intranuclear", "perinuclear", "nuclear edge", "foci", "protrusion"]
features_heatmap = df_heatmap.to_numpy(dtype=np.float32)
features_heatmap = np.round(features_heatmap, decimals=2)
print(features_heatmap.shape, features_heatmap.dtype)
df_heatmap

In [ ]:
def plot_heatmap_gene_masked(data, genes, patterns, figsize=(15, 5), colorbar=True, 
                             mask=None, annotations=None,
                             path_output=None, extension=None):
    # plot
    plt.figure(figsize=figsize)

    # heatmap
    ax = sns.heatmap(data=data, mask=mask, annot=annotations,
                     vmin=0, vmax=1, 
                     cmap="Reds", cbar=colorbar, 
                     xticklabels=patterns, yticklabels=genes)
    ax.set_facecolor("#f7fbff")

    # axes
    plt.ylim((23, 0))
    plt.xticks([])
    plt.yticks(rotation=0, fontsize=15, fontweight="bold")
    plt.tight_layout()

    # save frame    
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
                
    # show frame
    plt.show()
    
    return

In [ ]:
plot_heatmap_gene_masked(data=features_heatmap, 
                         genes=genes, 
                         patterns=patterns, 
                         figsize=(5, 10), 
                         colorbar=False,
                         mask=mask,
                         annotations=annotations,
                         path_output=os.path.join(plot_directory, "heatmap_genes_nocolorbar_masked"),
                         extension=["png", "pdf"])

In [ ]:
# collect predictions proportions
df_heatmap = df_30_no_drug.loc[df_30_no_drug.loc[:, "puromycin"] >= 0, :]
df_heatmap = df_heatmap.loc[:, ["cell", "key", 
                                "prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                "prediction_foci", "prediction_protrusion"]]
nb_cells_per_key = df_heatmap.groupby("key")["cell"].count()
df_heatmap = df_heatmap.groupby(by="key")["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                          "prediction_foci", "prediction_protrusion"].sum()
df_heatmap = df_heatmap.merge(right=nb_cells_per_key, how="inner", on="key")
df_heatmap.loc[:, "prediction_foci"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_protrusion"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_nuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_perinuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_intranuclear"] /= df_heatmap.loc[:, "cell"]

# sort genes
genes = ["MYH3", "MYH3_puro", "CEP192",
         "ATP6A2", "AP1S2", "AP1S2_puro", "AKAP9", "AKAP9_puro", "AKAP1", "AKAP1_puro", "HSP90B1", "HSP90B1_puro",
         "SPEN", "ASPM", "ASPM_puro",
         "DYNC1H1", "DYNC1H1_puro", "BUB1", "BUB1_puro", "CTNNB1", "CTNNB1_puro", 
         "HMMR", "HMMR_puro", "CEP170P1", "CRKL", "PAK2", "AURKA", "AURKA_puro",
         "KIF1C", "KIF1C_puro", "KIF4A", "KIF4A_puro", "RAB13", "DYNLL2", "KIF5B",
         "FLNA",
         "KIF20B", "MYO18A", "MYSNE2", "PLEC"]
genes_str = []
for gene in genes:
    if "_puro" in gene:
        gene_ = gene.split("_")[0]
        genes_str.append(gene_ + "*")
    else:
        genes_str.append(gene)
df_heatmap = df_heatmap.T.loc[:, genes]

# sort patterns
df_heatmap = df_heatmap.T
features_name = ["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                 "prediction_foci", "prediction_protrusion"]
df_heatmap = df_heatmap.loc[:, features_name]
print(df_heatmap.shape)

# build numpy matrix
patterns = ["intranuclear", "perinuclear", "nuclear edge", "foci", "protrusion"]
features_heatmap = df_heatmap.to_numpy(dtype=np.float32)
features_heatmap = np.round(features_heatmap, decimals=2)
print(features_heatmap.shape, features_heatmap.dtype)
df_heatmap

In [ ]:
def plot_heatmap_gene(data, genes, patterns, genes_str, figsize=(15, 5), colorbar=True,
                      path_output=None, extension=None):
    # plot
    plt.figure(figsize=figsize)

    # heatmap
    ax = sns.heatmap(data=data,
                     vmin=0, vmax=1, 
                     cmap="Reds", cbar=colorbar, 
                     xticklabels=patterns, yticklabels=genes)
    
    # axes
    plt.xticks([])
    plt.yticks(ticks=[i + 0.5 for i in range(0, 40)], labels=genes_str, rotation=0, fontsize=15, fontweight="bold")
    plt.ylim((40, 0))
    plt.tight_layout()

    # save frame    
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
                
    # show frame
    plt.show()
    
    return

In [ ]:
plot_heatmap_gene(data=features_heatmap, 
                  genes=genes, 
                  patterns=patterns,
                  genes_str=genes_str,
                  figsize=(5, 12), 
                  colorbar=False,
                  path_output=os.path.join(plot_directory, "heatmap_genes_all_nocolorbar"),
                  extension=["png", "pdf"])

In [ ]:
# collect predictions proportions
df_heatmap = df_30_no_drug.loc[df_30_no_drug.loc[:, "puromycin"] >= 0, :]
df_heatmap = df_heatmap.loc[:, ["cell", "key", 
                                "prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                "prediction_foci", "prediction_protrusion"]]
nb_cells_per_key = df_heatmap.groupby("key")["cell"].count()
df_heatmap = df_heatmap.groupby(by="key")["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                                          "prediction_foci", "prediction_protrusion"].sum()
df_heatmap = df_heatmap.merge(right=nb_cells_per_key, how="inner", on="key")
df_heatmap.loc[:, "prediction_foci"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_protrusion"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_nuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_perinuclear"] /= df_heatmap.loc[:, "cell"]
df_heatmap.loc[:, "prediction_intranuclear"] /= df_heatmap.loc[:, "cell"]

# sort genes
genes = ["MYH3", "MYH3_puro",
         "AP1S2", "AP1S2_puro", "AKAP9", "AKAP9_puro", "AKAP1", "AKAP1_puro", "HSP90B1", "HSP90B1_puro",
         "ASPM", "ASPM_puro",
         "DYNC1H1", "DYNC1H1_puro", "BUB1", "BUB1_puro", "CTNNB1", "CTNNB1_puro", 
         "HMMR", "HMMR_puro", "AURKA", "AURKA_puro",
         "KIF1C", "KIF1C_puro", "KIF4A", "KIF4A_puro"]
genes_str = []
for gene in genes:
    if "_puro" in gene:
        gene_ = gene.split("_")[0]
        genes_str.append(gene_ + "*")
    else:
        genes_str.append(gene)
df_heatmap = df_heatmap.T.loc[:, genes]

# sort patterns
df_heatmap = df_heatmap.T
features_name = ["prediction_intranuclear", "prediction_perinuclear", "prediction_nuclear", 
                 "prediction_foci", "prediction_protrusion"]
df_heatmap = df_heatmap.loc[:, features_name]
print(df_heatmap.shape)

# build numpy matrix
patterns = ["intranuclear", "perinuclear", "nuclear edge", "foci", "protrusion"]
features_heatmap = df_heatmap.to_numpy(dtype=np.float32)
features_heatmap = np.round(features_heatmap, decimals=2)
print(features_heatmap.shape, features_heatmap.dtype)
df_heatmap

In [ ]:
def plot_heatmap_gene(data, genes, patterns, genes_str, figsize=(15, 5), colorbar=True,
                      path_output=None, extension=None):
    # plot
    plt.figure(figsize=figsize)

    # heatmap
    ax = sns.heatmap(data=data,
                     vmin=0, vmax=1, 
                     cmap="Reds", cbar=colorbar, 
                     xticklabels=patterns, yticklabels=genes)
    
    # axes
    plt.xticks([])
    plt.yticks(ticks=[i + 0.5 for i in range(0, 26)], labels=genes_str, rotation=0, fontsize=15, fontweight="bold")
    plt.ylim((26, 0))
    plt.tight_layout()

    # save frame    
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
                
    # show frame
    plt.show()
    
    return

In [ ]:
plot_heatmap_gene(data=features_heatmap, 
                  genes=genes, 
                  patterns=patterns,
                  genes_str=genes_str,
                  figsize=(5, 10), 
                  colorbar=False,
                  path_output=os.path.join(plot_directory, "heatmap_genes_puro_nocolorbar"),
                  extension=["png", "pdf"])

In [ ]:
df_heatmap = df_30_no_drug.loc[df_30_no_drug.loc[:, "puromycin"] >= 0, :]
df_heatmap = df_heatmap.loc[:, ["cell", "key", 
                                "probability_intranuclear", "probability_perinuclear", "probability_nuclear", 
                                "probability_foci", "probability_protrusion"]]
print(df_heatmap.shape)
df_heatmap.head()

In [ ]:
def plot_heatmap_cell(data, key_name, patterns, figsize=(15, 5), colorbar=True,
                      path_output=None, extension=None):
    # plot
    plt.figure(figsize=figsize)

    # heatmap
    sns.heatmap(data=data,
                vmin=0, vmax=1, 
                cmap="Greens", cbar=colorbar, 
                xticklabels=patterns, yticklabels=False)
    
    # axes
    nb_cells = data.shape[0]
    if "puro" in key_name:
        key_name = key_name.split("_")[0] + "*"
    plt.xticks([])
    plt.yticks(fontsize=15, fontweight="bold")
    #plt.ylabel("{0} ({1} cells)".format(key_name, nb_cells), fontsize=20, fontweight="bold")
    plt.tight_layout()
    
    # save frame
    if path_output is not None and extension is not None:
        if isinstance(extension, str):
            path_output_ = path_output + "." + extension 
            plt.savefig(path_output_, format=extension, bbox_inches="tight", dpi="figure")
        elif isinstance(extension, list):
            for extension_ in extension:
                path_output_ = path_output + "." + extension_
                plt.savefig(path_output_, format=extension_, bbox_inches="tight", dpi="figure")
    
    # show frame
    plt.show()
    
    return

In [ ]:
predictions = ["probability_intranuclear", "probability_perinuclear", "probability_nuclear", 
               "probability_foci", "probability_protrusion"]

patterns = ["intranuclear", "perinuclear", "nuclear edge", "foci", "protrusion"]
keys = list(set(df_heatmap.loc[:, "key"]))
for key in keys:
    features_heatmap_key = df_heatmap.loc[df_heatmap.loc[:, "key"] == key, predictions].to_numpy(dtype=np.float32)
    features_heatmap_key = np.round(features_heatmap_key, decimals=2)
    linkage_matrix = linkage(features_heatmap_key, method='average', metric='euclidean', optimal_ordering=True)
    leaves_id = leaves_list(linkage_matrix)
    features_heatmap_key = features_heatmap_key[leaves_id, :]
    print(features_heatmap_key.shape, features_heatmap_key.dtype)
    plot_heatmap_cell(data=features_heatmap_key, 
                      key_name=key, 
                      patterns=patterns, 
                      figsize=(4, 8), 
                      colorbar=False,
                      path_output=os.path.join(plot_directory, "heatmap_cells_nocolorbar_{0}".format(key)),
                      extension=["png", "pdf"])

# Fisher tests

## Fisher test foci

In [ ]:
# parameters
genes = ["MYH3",
         "AP1S2", "AKAP9", "AKAP1", "HSP90B1",
         "ASPM",
         "DYNC1H1", "BUB1", "CTNNB1", 
         "HMMR", "AURKA",
         "KIF1C", "KIF4A"]
genes_puro = ["MYH3_puro",
              "AP1S2_puro", "AKAP9_puro", "AKAP1_puro", "HSP90B1_puro",
              "ASPM_puro",
              "DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", 
              "HMMR_puro", "AURKA_puro",
              "KIF1C_puro", "KIF4A_puro"]

prediction_foci = (df_30_no_drug.loc[:, "prediction_foci"]).astype(bool)
print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a foci pattern")
print("# - H0 %cell_puro == %cell")
print("# - H1 %cell_puro != %cell")
print("# - H0 rejected if p-value >= 0.05")
print("################################################", "\n")

# gene values
for i, gene in enumerate(genes):
    # no puromycin
    mask_gene = (df_30_no_drug.loc[:, "key"] == gene).astype(bool)
    prediction_foci_gene = prediction_foci[mask_gene]
    nb_control = len(prediction_foci_gene)
    nb_control_pattern = prediction_foci_gene.sum()
    nb_control_nopattern = nb_control - nb_control_pattern
    
    # puromycin
    gene_puro = genes_puro[i]
    mask_gene_puro = (df_30_no_drug.loc[:, "key"] == gene_puro).astype(bool)
    prediction_foci_gene_puro = prediction_foci[mask_gene_puro]
    nb_target = len(prediction_foci_gene_puro)
    nb_target_pattern = prediction_foci_gene_puro.sum()
    nb_target_nopattern = nb_target - nb_target_pattern
    
    # contingency table
    table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])

    # perform a two-sided Fisher test
    alpha = 0.05
    oddsratio, pvalue = fisher_exact(table, alternative="two-sided")

    proportion_target = (nb_target_pattern / nb_target) * 100
    proportion_control = (nb_control_pattern / nb_control) * 100

    print("{0} vs {1}".format(gene, gene_puro))
    print("  {0}: {1} cells | {2:.2f}% foci recognized".format(gene, nb_control, proportion_control))
    print("  {0}: {1} cells | {2:.2f}% foci recognized".format(gene_puro, nb_target, proportion_target))

    if pvalue > alpha:
        print("  => {0} not significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, gene, pvalue, oddsratio))
    else:
        print("  => {0} significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, gene, pvalue, oddsratio))
    print()

In [ ]:
# parameters
genes_puro = ["MYH3_puro",
              "AP1S2_puro", "AKAP9_puro", "AKAP1_puro", "HSP90B1_puro",
              "ASPM_puro",
              "DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", 
              "HMMR_puro", "AURKA_puro",
              "KIF1C_puro", "KIF4A_puro"]
genes_control = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

prediction_foci = (df_30_no_drug.loc[:, "prediction_foci"]).astype(bool)
print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a foci pattern")
print("# - H0 %cell_puro == %cell_control")
print("# - H1 %cell_puro != %cell_control")
print("# - H0 rejected if p-value >= 0.05")
print("#")
print("# Control genes: KIF20B, MYO18A, MYSNE2, PLEC")
print("################################################", "\n")

# mask control
mask_control = df_30_no_drug.loc[:, "gene"].isin(genes_control)
mask_not_annotated = ~df_30_no_drug.loc[:, "annotated"]
mask_control &= mask_not_annotated

# control values
prediction_foci_control = prediction_foci[mask_control]
nb_control = len(prediction_foci_control)
nb_control_pattern = prediction_foci_control.sum()
nb_control_nopattern = nb_control - nb_control_pattern

# gene values
for i, gene_puro in enumerate(genes_puro):
    # puromycin
    mask_gene_puro = (df_30_no_drug.loc[:, "key"] == gene_puro).astype(bool)
    prediction_foci_gene_puro = prediction_foci[mask_gene_puro]
    nb_target = len(prediction_foci_gene_puro)
    nb_target_pattern = prediction_foci_gene_puro.sum()
    nb_target_nopattern = nb_target - nb_target_pattern
    
    # contingency table
    table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])

    # perform a two-sided Fisher test
    alpha = 0.05
    oddsratio, pvalue = fisher_exact(table, alternative="two-sided")

    proportion_target = (nb_target_pattern / nb_target) * 100
    proportion_control = (nb_control_pattern / nb_control) * 100

    print("{0} vs {1}".format("Control", gene_puro))
    print("  {0}: {1} cells | {2:.2f}% foci recognized".format("Control", nb_control, proportion_control))
    print("  {0}: {1} cells | {2:.2f}% foci recognized".format(gene_puro, nb_target, proportion_target))

    if pvalue > alpha:
        print("  => {0} not significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, "Control", pvalue, oddsratio))
    else:
        print("  => {0} significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, "Control", pvalue, oddsratio))
    print()

In [ ]:
# parameters
dmso = "CTNNB1_DMSO"
lg007 = "CTNNB1_LG007"

prediction_foci = (df_30.loc[:, "prediction_foci"]).astype(bool)

print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a foci pattern")
print("# - H0 %DMSO == %LG007")
print("# - H1 %DMSO != %LG007")
print("# - H0 rejected if p-value >= 0.05")
print("################################################", "\n")

# masks
mask_dmso = (df_30.loc[:, "key"] == dmso).astype(bool)
mask_lg007 = (df_30.loc[:, "key"] == lg007).astype(bool)

# values DMSO
prediction_foci_dmso = prediction_foci[mask_dmso]
nb_control = len(prediction_foci_dmso)
nb_control_pattern = prediction_foci_dmso.sum()
nb_control_nopattern = nb_control - nb_control_pattern

# values LG007
prediction_foci_lg007 = prediction_foci[mask_lg007]
nb_target = len(prediction_foci_lg007)
nb_target_pattern = prediction_foci_lg007.sum()
nb_target_nopattern = nb_target - nb_target_pattern

# contingency table
table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])

# perform a two-sided Fisher test
alpha = 0.05
oddsratio, pvalue = fisher_exact(table, alternative="two-sided")

proportion_target = (nb_target_pattern / nb_target) * 100
proportion_control = (nb_control_pattern / nb_control) * 100

print("{0} vs {1}".format(dmso, lg007))
print("  {0}: {1} cells | {2:.2f}% foci recognized".format(dmso, nb_control, proportion_control))
print("  {0}: {1} cells | {2:.2f}% foci recognized".format(lg007, nb_target, proportion_target))

if pvalue > alpha:
    print("  => {0} not significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
          .format(lg007, dmso, pvalue, oddsratio))
else:
    print("  => {0} significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
          .format(lg007, dmso, pvalue, oddsratio))

## Fisher test protrusion

In [ ]:
# parameters
genes = ["MYH3",
         "AP1S2", "AKAP9", "AKAP1", "HSP90B1",
         "ASPM",
         "DYNC1H1", "BUB1", "CTNNB1", 
         "HMMR", "AURKA",
         "KIF1C", "KIF4A"]
genes_puro = ["MYH3_puro",
              "AP1S2_puro", "AKAP9_puro", "AKAP1_puro", "HSP90B1_puro",
              "ASPM_puro",
              "DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", 
              "HMMR_puro", "AURKA_puro",
              "KIF1C_puro", "KIF4A_puro"]

prediction_foci = (df_30_no_drug.loc[:, "prediction_protrusion"]).astype(bool)

print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a protrusion pattern")
print("# - H0 %cell_puro == %cell")
print("# - H1 %cell_puro != %cell")
print("# - H0 rejected if p-value >= 0.05")
print("################################################", "\n")

# gene values
for i, gene in enumerate(genes):
    # no puromycin
    mask_gene = (df_30_no_drug.loc[:, "key"] == gene).astype(bool)
    prediction_foci_gene = prediction_foci[mask_gene]
    nb_control = len(prediction_foci_gene)
    nb_control_pattern = prediction_foci_gene.sum()
    nb_control_nopattern = nb_control - nb_control_pattern
    
    # puromycin
    gene_puro = genes_puro[i]
    mask_gene_puro = (df_30_no_drug.loc[:, "key"] == gene_puro).astype(bool)
    prediction_foci_gene_puro = prediction_foci[mask_gene_puro]
    nb_target = len(prediction_foci_gene_puro)
    nb_target_pattern = prediction_foci_gene_puro.sum()
    nb_target_nopattern = nb_target - nb_target_pattern
    
    # contingency table
    table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])

    # perform a two-sided Fisher test
    alpha = 0.05
    oddsratio, pvalue = fisher_exact(table, alternative="two-sided")

    proportion_target = (nb_target_pattern / nb_target) * 100
    proportion_control = (nb_control_pattern / nb_control) * 100

    print("{0} vs {1}".format(gene, gene_puro))
    print("  {0}: {1} cells | {2:.2f}% protrusion recognized".format(gene, nb_control, proportion_control))
    print("  {0}: {1} cells | {2:.2f}% protrusion recognized".format(gene_puro, nb_target, proportion_target))

    if pvalue > alpha:
        print("  => {0} not significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, gene, pvalue, oddsratio))
    else:
        print("  => {0} significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, gene, pvalue, oddsratio))
    print()

In [ ]:
# parameters
genes_puro = ["MYH3_puro",
              "AP1S2_puro", "AKAP9_puro", "AKAP1_puro", "HSP90B1_puro",
              "ASPM_puro",
              "DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", 
              "HMMR_puro", "AURKA_puro",
              "KIF1C_puro", "KIF4A_puro"]
genes_control = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

prediction_foci = (df_30_no_drug.loc[:, "prediction_protrusion"]).astype(bool)

print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a protrusion pattern")
print("# - H0 %cell_puro == %cell_control")
print("# - H1 %cell_puro != %cell_control")
print("# - H0 rejected if p-value >= 0.05")
print("#")
print("# Control genes: KIF20B, MYO18A, MYSNE2, PLEC")
print("################################################", "\n")

# mask control
mask_control = df_30_no_drug.loc[:, "gene"].isin(genes_control)
mask_not_annotated = ~df_30_no_drug.loc[:, "annotated"]
mask_control &= mask_not_annotated

# control values
prediction_foci_control = prediction_foci[mask_control]
nb_control = len(prediction_foci_control)
nb_control_pattern = prediction_foci_control.sum()
nb_control_nopattern = nb_control - nb_control_pattern

# gene values
for i, gene_puro in enumerate(genes_puro):
    # puromycin
    mask_gene_puro = (df_30_no_drug.loc[:, "key"] == gene_puro).astype(bool)
    prediction_foci_gene_puro = prediction_foci[mask_gene_puro]
    nb_target = len(prediction_foci_gene_puro)
    nb_target_pattern = prediction_foci_gene_puro.sum()
    nb_target_nopattern = nb_target - nb_target_pattern
    
    # contingency table
    table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])

    # perform a two-sided Fisher test
    alpha = 0.05
    oddsratio, pvalue = fisher_exact(table, alternative="two-sided")

    proportion_target = (nb_target_pattern / nb_target) * 100
    proportion_control = (nb_control_pattern / nb_control) * 100

    print("{0} vs {1}".format("Control", gene_puro))
    print("  {0}: {1} cells | {2:.2f}% protrusion recognized".format("Control", nb_control, proportion_control))
    print("  {0}: {1} cells | {2:.2f}% protrusion recognized".format(gene_puro, nb_target, proportion_target))

    if pvalue > alpha:
        print("  => {0} not significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, "Control", pvalue, oddsratio))
    else:
        print("  => {0} significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, "Control", pvalue, oddsratio))
    print()

## Fisher test perinuclear

In [ ]:
# parameters
genes = ["MYH3",
         "AP1S2", "AKAP9", "AKAP1", "HSP90B1",
         "ASPM",
         "DYNC1H1", "BUB1", "CTNNB1", 
         "HMMR", "AURKA",
         "KIF1C", "KIF4A"]
genes_puro = ["MYH3_puro",
              "AP1S2_puro", "AKAP9_puro", "AKAP1_puro", "HSP90B1_puro",
              "ASPM_puro",
              "DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", 
              "HMMR_puro", "AURKA_puro",
              "KIF1C_puro", "KIF4A_puro"]

prediction_foci = (df_30_no_drug.loc[:, "prediction_perinuclear"]).astype(bool)

print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a perinuclear pattern")
print("# - H0 %cell_puro == %cell")
print("# - H1 %cell_puro != %cell")
print("# - H0 rejected if p-value >= 0.05")
print("################################################", "\n")

# gene values
for i, gene in enumerate(genes):
    # no puromycin
    mask_gene = (df_30_no_drug.loc[:, "key"] == gene).astype(bool)
    prediction_foci_gene = prediction_foci[mask_gene]
    nb_control = len(prediction_foci_gene)
    nb_control_pattern = prediction_foci_gene.sum()
    nb_control_nopattern = nb_control - nb_control_pattern
    
    # puromycin
    gene_puro = genes_puro[i]
    mask_gene_puro = (df_30_no_drug.loc[:, "key"] == gene_puro).astype(bool)
    prediction_foci_gene_puro = prediction_foci[mask_gene_puro]
    nb_target = len(prediction_foci_gene_puro)
    nb_target_pattern = prediction_foci_gene_puro.sum()
    nb_target_nopattern = nb_target - nb_target_pattern
    
    # contingency table
    table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])

    # perform a two-sided Fisher test
    alpha = 0.05
    oddsratio, pvalue = fisher_exact(table, alternative="two-sided")

    proportion_target = (nb_target_pattern / nb_target) * 100
    proportion_control = (nb_control_pattern / nb_control) * 100

    print("{0} vs {1}".format(gene, gene_puro))
    print("  {0}: {1} cells | {2:.2f}% perinuclear recognized".format(gene, nb_control, proportion_control))
    print("  {0}: {1} cells | {2:.2f}% perinuclear recognized".format(gene_puro, nb_target, proportion_target))

    if pvalue > alpha:
        print("  => {0} not significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, gene, pvalue, oddsratio))
    else:
        print("  => {0} significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, gene, pvalue, oddsratio))
    print()

In [ ]:
# parameters
genes_puro = ["MYH3_puro",
              "AP1S2_puro", "AKAP9_puro", "AKAP1_puro", "HSP90B1_puro",
              "ASPM_puro",
              "DYNC1H1_puro", "BUB1_puro", "CTNNB1_puro", 
              "HMMR_puro", "AURKA_puro",
              "KIF1C_puro", "KIF4A_puro"]
genes_control = ["KIF20B", "MYO18A", "MYSNE2", "PLEC"]

prediction_foci = (df_30_no_drug.loc[:, "prediction_perinuclear"]).astype(bool)

print("################################################")
print("# Fisher exact test:")
print("# - Number of cells classify with a perinuclear pattern")
print("# - H0 %cell_puro == %cell_control")
print("# - H1 %cell_puro != %cell_control")
print("# - H0 rejected if p-value >= 0.05")
print("#")
print("# Control genes: KIF20B, MYO18A, MYSNE2, PLEC")
print("################################################", "\n")

# mask control
mask_control = df_30_no_drug.loc[:, "gene"].isin(genes_control)
mask_not_annotated = ~df_30_no_drug.loc[:, "annotated"]
mask_control &= mask_not_annotated

# control values
prediction_foci_control = prediction_foci[mask_control]
nb_control = len(prediction_foci_control)
nb_control_pattern = prediction_foci_control.sum()
nb_control_nopattern = nb_control - nb_control_pattern

# gene values
for i, gene_puro in enumerate(genes_puro):
    # puromycin
    mask_gene_puro = (df_30_no_drug.loc[:, "key"] == gene_puro).astype(bool)
    prediction_foci_gene_puro = prediction_foci[mask_gene_puro]
    nb_target = len(prediction_foci_gene_puro)
    nb_target_pattern = prediction_foci_gene_puro.sum()
    nb_target_nopattern = nb_target - nb_target_pattern
    
    # contingency table
    table = np.array([[nb_target_pattern, nb_target_nopattern], [nb_control_pattern, nb_control_nopattern]])

    # perform a two-sided Fisher test
    alpha = 0.05
    oddsratio, pvalue = fisher_exact(table, alternative="two-sided")

    proportion_target = (nb_target_pattern / nb_target) * 100
    proportion_control = (nb_control_pattern / nb_control) * 100

    print("{0} vs {1}".format("Control", gene_puro))
    print("  {0}: {1} cells | {2:.2f}% perinuclear recognized".format("Control", nb_control, proportion_control))
    print("  {0}: {1} cells | {2:.2f}% perinuclear recognized".format(gene_puro, nb_target, proportion_target))

    if pvalue > alpha:
        print("  => {0} not significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, "Control", pvalue, oddsratio))
    else:
        print("  => {0} significantly different than {1} (p-value:{2} | odd-ratio:{3:.3f})"
              .format(gene_puro, "Control", pvalue, oddsratio))
    print()